# **Retail Energy and Emissions calculator** 

There is an urgent need to understand and quantify the energy and emissions brought by changes in distribution channels that enable growth, and the changes in demand fueling such growth. This will provide a fundamental understanding for the development or identification of pathways to prioritize investments and interventions (e.g., public and private sector policies or strategies) that allow decoupling economic growth from emissions and energy. This is key to achieve the much-needed reductions in net carbon emissions and mitigate global warming.

The objective of this proposed study is to quantify energy and emissions (in the form of efficiency rates) in retail distribution through the development and design of several scenarios. In summary, the scenarios will focus on i) consumption trends based on recent changes in consumer behavior and their choice of retail channel; ii) new distribution network design and capacity; iii) changes in delivery modes and vehicle technologies; and iv) public and private decarbonization efforts.

# 0. Packages

In [1]:
# ------------------------------------load all necessary libraries ---------------------------------------------- #
# Mat and DataFrame
import numpy as np # For vectorized math operations
import pandas as pd # Dataframe
import os
# Figures
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Random function
import random 
import seaborn as sns

# 1. Inputs

## 1.1 Files

All the files required to run this model are in the next link. Two folders are considered, the working folder "GenIMLLP_model" and previous models "NCST"

Link: https://ucdavis.box.com/s/mhpvmvgn9db75kbmfm6lql7yzo8wvqmv

## 1.2. Enter all the parameters 

Each scenario result as a combination of parameters that should be defined in the section "DATA REQUIRED TO RUN THE CODE" In[2] in the model "Model-5-23.ipynb". 
Bellow you can find the inputs specifications. 

**Base Purchasing Scenarios**

There are currently three high level scenarios in the model (and another, default, as a placeholder). While most of the values used are the same across all scenarios (and are by default selected considering the most common scenario), we detail below the most important scenario assumptions used in the model, and the rationale for the assumptions selected in the given base scenarios. Please find the csv files in the data folder. A file call example.xls with information of possible inputs are also in the same folder.

1. Retail Scenario
2. E-tail Scenario
3. Efficient e-tail Scenario
4. Both Scenario

**Metropolitan Statistical Areas (MSA)**

1. Chicago (CHI)
2. Washington D.C. (DC)
3. Los Angeles (LA)
4. New York City (NYC)
5. San Francisco (SF) 

**Population growth scenarios specification**

We attribute the future population growth to two developments, one is the future change in the socioeconomic conditions of the region, reflected in the change in the market shares of the different independent variables, and the other is the future growth of the total population of the region. We delineate two growth patterns, a moderate pattern, which implies little change in future socioeconomic conditions from the present and no significant growth in total population, and a high growth pattern, which implies a significant growth in both socioeconomic conditions and total population projection. After permutation, Table 2 shows the four population growth scenarios that we have delineated. 

1. Moderate-Moderate (MM)
2. Moderate-High growth (MH)
3. High growth-Moderate (HM)
4. High growth-High growth (HH)

**Energy and emission scenario**

User should provide here which scenario of Energy Intensity and Emission per year per vehicle want to consider: 

1. BAU - MOVES model
2. BAU - Energy Information Administration - EIA
3. NZE50 - Net Zero 2050 - IEA
4. Custom: Energy_Percentages.csv

**Year analyzed in the tornado diagram**

User has the option to change the year that want to analyze in the one direction sensitivity analysis.

**Path for running**

Paste the path of the working folder and the path of the NCST data folder.


In [2]:
#----------------------------------- DATA REQUIRED TO RUN THE CODE ---------------------------------------------------- #

# Base Purchasing Scenarios: 1. Retail, 2. E-com, 3. Efficient e-tail, 4. Omnichannel
option = 1

# MSA - Enter the MSA number: 1. CHI, 2. DC, 3. LA, 4. NYC, 5. SF
MSA_op = 4

#Enter the market share and population growth scenario: MM, MH, HM, HH:
PG = "MM"

# Years analyzed
Years = np.array([2020,2030,2040,2050])

# Energy and emission scenario: 1. BAU - MOVES, 2. BAU - EIA, 3. NZE50, 4. Custom
Energy_option = 1

# Year analyzed - Tornado Diagram
Year_Tornado = 2020

# Path - Copy the path of your working folder
Path = 'C:/Users/cata1/Box/Catalina Valencia/Project 1 - nDEE/Code/GenIMLLP_model'

# Copy the path of NCST data folder
Path2 = 'C:/Users/cata1/Box/Catalina Valencia/Project 1 - nDEE/Code/NCST/data'

## 1.2. Read the Base Purchasing Scenarios (Retail, E-com, Efficient e-etail, Both(Retail-E-com))**

In [3]:
def Purchasing_Scenarios(option):

    global Title
    
    # File name
    if option == "1":
        file = '/Data/Retail_Senario.csv'
        Title = "Retail"
    elif option == "2":
        file = '/Data/E_tail_Scenario.csv'
        Title = "Etail"
    elif option == "3":
        file = '/Data/EE_tail_Scenario.csv'
        Title = "EEtail"
    elif option == "4":
        file = '/Data/Both_Scenario.csv'
        Title = "Retail and Etail"
    
    # Output
    path_name = Path + file
    Scenario = pd.read_csv(path_name, sep=',', header=0)  
    return(Scenario)


## 1.3. Read the Metropolitan Statistical Areas (MSA) and Population growth scenarios specification

In [4]:
# Market Share and Population characteristics
def MSA_Market_Population(MSA_op):
    
    # File name
    if MSA_op == "1":
        file = '/Data/ARIMA_CHI.csv'
    elif MSA_op == "2":
        file = '/Data/ARIMA_DC.csv'
    elif MSA_op == "3":
        file = '/Data/ARIMA_LA.csv'
    elif MSA_op == "4":
        file = '/Data/ARIMA_NYC.csv'
    elif MSA_op == "5":
        file = '/Data/ARIMA_SF.csv'
  
    # Read the file
    path_name = Path + file
    MSA = pd.read_csv(path_name, sep=',', header=0)  

    # Concatenate market share and population growth scenario
    MSA['Concatenate'] = MSA['Market'] + MSA['Population']
    MSA.set_index('Concatenate', inplace=True)

    # Market population matrix
    Get_selection = MSA.loc[PG]  
    Get_selection = Get_selection.reset_index(drop=True)
    
    # Output
    Market_Population = pd.DataFrame(Get_selection)
    Market_Population.columns=['Market','Population','Year','No shopping','In-store','Online','Both']
    Years = Market_Population['Year']
    Market_Population.set_index('Year', inplace=True) 
    return(Market_Population)


## 1.4. Energy and emission scenario


In [5]:
def Rates(Energy_option):
    # Read the Summary_Report_Rates from MOVES model
    path_name = Path + '/Data/Summary_Report_Rates.csv'
    SRR = pd.read_csv(path_name, sep=',', header=0, na_values=['#DIV/0!'])
    
    # null value 
    SRR.isnull().sum()
    SRR[SRR != 0]

    # --------------------------------- Check the MSA from the State ----------------------------------------- #
    States_cities = [[17, 11, 6, 36, 6],[1, 2, 3, 4, 5]] # Considering state ID of the report
    for i in range(len(States_cities[0])):
        if States_cities[1][i] == int(MSA_op):
            State = States_cities[0][i]

    #  ---------------------------- Get the Total Energy from summary report rate ---------------------------- #
    # Mean-Energy per state
    SRR_state = SRR.loc[SRR['State'] == State]
    Report_Energy1 = (SRR_state.groupby(['Source','Year'])['TotalEnergy'].mean().unstack())/1000 #MJ/mile
    #Report_
    # Total energy per state
    global Report_Energy_Total
    Report_Energy_Total = (SRR_state.groupby(['Source','Year'])['TotalEnergy'].sum().unstack())/1000 #MJ/mile  
    # Mean-Miles per vehicle type
    global Report_Distance
    Report_Distance = (SRR_state.groupby(['Source','Year'])['Distance'].mean().unstack()) 
    # Vehicle class
    Vehicle_class = [21, 31, 32, 41, 42, 52, 53, 61, 62]
    Len_vehicles = len(Vehicle_class)
    Len_years = len(Years)
    # ---------------------------------------- glbal variables ------------------------------------------------ #
    global Report_Energy
    global Electricity_share_of_ETrucks
    global CO2_Aviation
    global Electricity_share_of_Commercial
    global Electricity_share_of_EV
    global Energy_Percentage_table
    global MJ_Aviation
    global CO2_per_MJ_Electricity
    global MJ_Paper
    global CO2_Paper
    global MJ_Warehouse
    global CO2_Warehouse
    global NOX_per_MJ
    global PM25_per_MJ
    global SO2_per_MJ
    global NOX_Aviation
    global PM25_Aviation
    global SO2_per_MJ
    global NOX_Paper
    global PM25_Paper
    global SO2_Paper
    global NOX_Warehouse
    global PM25_Warehouse
    global SO2_Warehouse
    Energy_Percentage_table = []

    # ----------------------- Choose an option to consider the energy and emission scenario ------------------ #
    # MOVES Energy rates
    if Energy_option == "1": 
        Report_Energy = Report_Energy1.copy()
        # To Print the percentage
        Per = np.zeros((int(Len_years)-1,int(Len_vehicles)))
        for i in range(Len_years-1):
            for j in range(Len_vehicles):
                Per[i][j] = (Report_Energy[Years[0]][Vehicle_class[j]]/Report_Energy[Years[i+1]][Vehicle_class[j]])-1
        Percentages = np.transpose(Per, axes=None)*100
        Energy_Percentage_table = pd.DataFrame(Percentages, columns=['2030', '2040', '2050']) #Years_percentage#
    
    # BAU - EIA
    elif Energy_option == "2": 
        file = Path + '/Data/Summary_Report_Rates_BAU_EIA.csv'
        Percentages = pd.read_csv(file, sep=',', header=0) 
        Per_energy = np.array(Percentages.loc[0:8,'2020':'2050'])
        Report_Energy = Report_Energy1.copy()
        j=0
        for i in Years:
            Report_Energy[i] = Report_Energy1[2020]*Per_energy[:,j]
            j=j+1
        Per_reduction = np.ones((9,len(Years)))*100-Percentages.loc[0:8,'2020':'2050']*100        
        Energy_Percentage_table = pd.DataFrame(Per_reduction)
        Energy_Percentage_table.columns=['2020','2030', '2040', '2050']

        # Electricity share of EV
        New_ETrucks = np.array(Percentages.loc[9,'2020':'2050'])
        Electricity_share_of_ETrucks = pd.DataFrame(New_ETrucks, index = ['2020', '2030', '2040', '2050'])
        New_EV = np.array(Percentages.loc[10,'2020':'2050'])
        Electricity_share_of_EV = pd.DataFrame(New_EV, index = ['2020', '2030', '2040', '2050'])
        New_Commercial = np.array(Percentages.loc[11,'2020':'2050'])
        Electricity_share_of_Commercial = pd.DataFrame(New_Commercial, index = ['2020', '2030', '2040', '2050'])

        # Air
        New_CO2Air = np.array(Percentages.loc[12,'2020':'2050'])
        CO2_Aviation = pd.DataFrame(New_CO2Air, index = ['2020', '2030', '2040', '2050'])
        New_MJAir = np.array(Percentages.loc[13,'2020':'2050'])
        MJ_Aviation = pd.DataFrame(New_MJAir, index = ['2020', '2030', '2040', '2050'])

        # Electricity
        New_CO2 = np.array(Percentages.loc[14+int(MSA_op),'2020':'2050'])
        CO2_per_MJ_Electricity = pd.DataFrame(New_CO2 , index = ['2020', '2030', '2040', '2050'])
        
        # Paper_Industry
        New_MJP = np.array(Percentages.loc[20,'2020':'2050'])
        MJ_Paper = pd.DataFrame(New_MJP, index = ['2020', '2030', '2040', '2050'])
        New_CO2P = np.array(Percentages.loc[21,'2020':'2050'])
        CO2_Paper = pd.DataFrame(New_CO2P, index = ['2020', '2030', '2040', '2050'])
        
        # Warehouse
        New_MJW = np.array(Percentages.loc[22,'2020':'2050'])
        MJ_Warehouse = pd.DataFrame(New_MJW, index = ['2020', '2030', '2040', '2050'])
        New_CO2W = np.array(Percentages.loc[23,'2020':'2050'])
        CO2_Warehouse = pd.DataFrame(New_CO2W, index = ['2020', '2030', '2040', '2050'])
    
    # Net Zero Emissions by 2050    
    elif Energy_option == "3": 
        file = Path + '/Data/Summary_Report_Rates_NZE50.csv'
        Percentages = pd.read_csv(file, sep=',', header=0) 
        Per_energy = np.array(Percentages.loc[0:8,'2020':'2050'])
        Report_Energy = Report_Energy1.copy()
        j=0
        for i in Years:
            Report_Energy[i] = Report_Energy1[2020]*Per_energy[:,j]
            j=j+1

        Per_reduction = np.ones((9,len(Years)))*100-Percentages.loc[0:8,'2020':'2050']*100        
        Energy_Percentage_table = pd.DataFrame(Per_reduction)
        Energy_Percentage_table.columns=['2020','2030', '2040', '2050']

        # Electricity share of EV
        New_ETrucks = np.array(Percentages.loc[9,'2020':'2050'])
        Electricity_share_of_ETrucks = pd.DataFrame(New_ETrucks, index = ['2020', '2030', '2040', '2050'])
        New_EV = np.array(Percentages.loc[10,'2020':'2050'])
        Electricity_share_of_EV = pd.DataFrame(New_EV, index = ['2020', '2030', '2040', '2050'])
        New_Commercial = np.array(Percentages.loc[11,'2020':'2050'])
        Electricity_share_of_Commercial = pd.DataFrame(New_Commercial, index = ['2020', '2030', '2040', '2050'])

        # CO2 emissions for Air
        New_CO2Air = np.array(Percentages.loc[12,'2020':'2050'])
        CO2_Aviation = pd.DataFrame(New_CO2Air, index = ['2020', '2030', '2040', '2050'])
        New_MJAir = np.array(Percentages.loc[13,'2020':'2050'])
        MJ_Aviation = pd.DataFrame(New_MJAir, index = ['2020', '2030', '2040', '2050'])

        # CO2 emissions per MJ Electricity
        New_CO2 = np.array(Percentages.loc[14,'2020':'2050'])
        CO2_per_MJ_Electricity = pd.DataFrame(New_CO2 , index = ['2020', '2030', '2040', '2050'])
        
        # Paper_Industry
        New_MJP = np.array(Percentages.loc[20,'2020':'2050'])
        MJ_Paper = pd.DataFrame(New_MJP, index = ['2020', '2030', '2040', '2050'])
        New_CO2P = np.array(Percentages.loc[21,'2020':'2050'])
        CO2_Paper = pd.DataFrame(New_CO2P, index = ['2020', '2030', '2040', '2050'])
        
        # Warehouse
        New_MJW = np.array(Percentages.loc[22,'2020':'2050'])
        MJ_Warehouse = pd.DataFrame(New_MJW, index = ['2020', '2030', '2040', '2050'])
        New_CO2W = np.array(Percentages.loc[23,'2020':'2050'])
        CO2_Warehouse = pd.DataFrame(New_CO2W, index = ['2020', '2030', '2040', '2050'])
    
    # For entering your own rates   
    elif Energy_option == "4": 
        file = Path + '/Data/Summary_Report_Rates_Custom.csv'
        Percentages = pd.read_csv(file, sep=',', header=0) 
        Energy_Percentage_table = pd.DataFrame(Percentages) 
        Report_Energy = Report_Energy1.copy()
        for i in Years:
            if (Report_Energy[i] > 2020).all():
                Report_Energy[i] = Report_Energy1[2020]*Percentages[Years[i].astype(str)]/100

    # Print the percentage per vehicle per year
    Title1 = "Percentage of Reduction in Energy"
    Title2 = "Energy MJ/mi"


    if (Energy_option == "1" or Energy_option == "4"):
        # Electricity share of EV
        ones = [1,1,1,1]
        Electricity_share_of_ETrucks = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        Electricity_share_of_EV = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        Electricity_share_of_Commercial = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        
        # Electricity
        CO2_per_MJ_Electricity = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        NOX_per_MJ = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        PM25_per_MJ = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        SO2_per_MJ = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        
        # Air
        MJ_Aviation = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        CO2_Aviation = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        NOX_Aviation = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        PM25_Aviation = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        SO2_Aviation = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
       
        # Paper_Industry
        MJ_Paper = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        CO2_Paper = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        NOX_Paper = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        PM25_Paper = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        SO2_Paper = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        
        # Warehouse
        MJ_Warehouse = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        CO2_Warehouse = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        NOX_Warehouse = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        PM25_Warehouse = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])
        SO2_Warehouse = pd.DataFrame(ones, index = ['2020', '2030', '2040', '2050'])

    
    return


## 1.5. Scenario with the level selected 

Inputs are defined as variables according to the level selected

In [6]:
# -------------------------------------- Scenario select level ---------------------------------------------- #
  
def Scenario_Level(Scenario):
    global Average_Weigth_Item
    Average_Weigth_Item = []
    if Scenario.Select_Level[0] == 'low':
        Average_Weigth_Item = Scenario.low[0]
    elif Scenario.Select_Level[0] == 'med':
        Average_Weigth_Item = Scenario.med[0]
    else: 
        Average_Weigth_Item = Scenario.high[0]

    # Weight of each shipping package
    global Weight_shipping_package
    Weight_shipping_package = []
    if Scenario.Select_Level[1] == 'low':
        Weight_shipping_package = Scenario.low[1]
    elif Scenario.Select_Level[1] == 'med':
        Weight_shipping_package = Scenario.med[1]
    else:
        Weight_shipping_package = Scenario.high[1]

    # Return Rates from Online Purchase 
    global Rate_Online_Purchase
    Rate_Online_Purchase = []
    if Scenario.Select_Level[2] == 'low':
        Rate_Online_Purchase = Scenario.low[2] 
    elif Scenario.Select_Level[2] == 'med':
        Rate_Online_Purchase = Scenario.med[2] 
    else:
        Rate_Online_Purchase = Scenario.high[2] 

    # Return Rates from In-store Purchases 
    global Rate_Instore_Purchase
    Rate_Instore_Purchase = []
    if Scenario.Select_Level[3] == 'low':
        Rate_Instore_Purchase = Scenario.low[3] 
    elif Scenario.Select_Level[3] == 'med':
        Rate_Instore_Purchase = Scenario.med[3] 
    else:
        Rate_Instore_Purchase = Scenario.high[3]
    # Average Cost per item
    global Cost_per_item
    Cost_per_item = []
    if Scenario.Select_Level[4] == 'low':
        Cost_per_item = Scenario.low[4]
    elif Scenario.Select_Level[4] == 'med':
        Cost_per_item = Scenario.med[4]
    else:
        Cost_per_item = Scenario.high[4]

    #------------------------------------------------- Packaging ------------------------------------------------ #
    # Average item volume
    global Average_item_volume
    Average_item_volume = []
    if Scenario.Select_Level[5] == 'low':
        Average_item_volume = Scenario.low[5]
    elif Scenario.Select_Level[5] == 'med':
        Average_item_volume = Scenario.med[5]
    else: 
        Average_item_volume = Scenario.high[5]

    # Energy & Emission per packaging material (Retail)
    global Energy_Emission_per_packaging_material_Retail
    Energy_Emission_per_packaging_material_Retail = []
    if Scenario.Select_Level[6] == 'low':
        Energy_Emission_per_packaging_material_Retail = 0
    elif Scenario.Select_Level[6] == 'med':
        Energy_Emission_per_packaging_material_Retail = 1
    else: 
        Energy_Emission_per_packaging_material_Retail = 2

    # Energy & Emission per packaging material (Etail)
    global Energy_Emission_per_packaging_material_Etail
    Energy_Emission_per_packaging_material_Etail = []
    if Scenario.Select_Level[7] == 'low':
        Energy_Emission_per_packaging_material_Etail = 0
    elif Scenario.Select_Level[7] == 'med':
        Energy_Emission_per_packaging_material_Etail = 1
    else: 
        Energy_Emission_per_packaging_material_Etail = 2

    # -------------------------------------------- Shipping Option ---------------------------------------------- #
    # Select Level of Rush Shipments
    global Select_Level_of_Rush_Shipments
    Select_Level_of_Rush_Shipments = []
    if Scenario.Select_Level[8] == 'low':
        Select_Level_of_Rush_Shipments = 0
    elif Scenario.Select_Level[8] == 'med':
        Select_Level_of_Rush_Shipments = 1
    else: 
        Select_Level_of_Rush_Shipments = 2

    # Air %
    global Energy_Intensity_Air_perc
    Energy_Intensity_Air_perc = [] 
    if Scenario.Select_Level[9] == 'low':
        Energy_Intensity_Air_perc = Scenario.low[9] 
    elif Scenario.Select_Level[9] == 'med':
        Energy_Intensity_Air_perc = Scenario.med[9] 
    else: 
        Energy_Intensity_Air_perc = Scenario.high[9] 

    # -------------------------------------------- Energy Intensity ---------------------------------------------- #

    # Rail
    global Energy_Intensity_Rail
    Energy_Intensity_Rail = [] 
    if Scenario.Select_Level[10] == 'low':
        Energy_Intensity_Rail = Scenario.low[10]
    elif Scenario.Select_Level[10] == 'med':
        Energy_Intensity_Rail = Scenario.med[10]
    else: 
        Energy_Intensity_Rail = Scenario.high[10]  

    # Air Carrier Shipment 
    global Energy_Intensity_Air
    Energy_Intensity_Air = [] 
    if Scenario.Select_Level[11] == 'low':
        Energy_Intensity_Air = Scenario.low[11]
    elif Scenario.Select_Level[11] == 'med':
        Energy_Intensity_Air = Scenario.med[11]
    else: 
        Energy_Intensity_Air = Scenario.high[11]    

    # ------------------------------------------------ Freight ---------------------------------------------------- #
    # Select type of road vehicle for FTL
    global Type_of_road_vehicle_FTL
    Type_of_road_vehicle_FTL = Scenario.Select_Level[12] 

    # Select Shipping efficiency
    global Select_Shipping_efficiency
    Select_Shipping_efficiency = []
    if Scenario.Select_Level[13] == 'low':
        Select_Shipping_efficiency = 2
    elif Scenario.Select_Level[13] == 'med':
        Select_Shipping_efficiency = 1
    else: 
        Select_Shipping_efficiency = 0  

    # --------------------------------------- Computer usage ----------------------------------------------------- #
    # Energy Usage Selection
    global Energy_Usage_Selection
    Energy_Usage_Selection = [] # kWh/GB
    if Scenario.Select_Level[15] == 'low':
        Energy_Usage_Selection = Scenario.low[15]
    elif Scenario.Select_Level[15] == 'med':
        Energy_Usage_Selection = Scenario.med[15]
    else: 
        Energy_Usage_Selection = Scenario.high[15]

    # Data usage per item
    global Data_Usage_Item
    Data_Usage_Item = [] #MB/item
    if Scenario.Select_Level[16] == 'low':
        Data_Usage_Item = Scenario.low[16]
    elif Scenario.Select_Level[16] == 'med':
        Data_Usage_Item = Scenario.med[16]
    else: 
        Data_Usage_Item = Scenario.high[16]

    # ------------------------------------------ Customer Transport ---------------------------------------------- #
    # Basket size in-store
    global Basket_Size_InStore
    Basket_Size_InStore = [] # items/person
    if Scenario.Select_Level[17] == 'low':
        Basket_Size_InStore = Scenario.low[17]
    elif Scenario.Select_Level[17] == 'med':
        Basket_Size_InStore = Scenario.med[17]
    else: 
        Basket_Size_InStore = Scenario.high[17]

    # Basket size online
    global Basket_size_online
    Basket_size_online = [] # items/order
    if Scenario.Select_Level[18] == 'low':
        Basket_size_online = Scenario.low[18]
    elif Scenario.Select_Level[18] == 'med':
        Basket_size_online = Scenario.med[18]
    else: 
        Basket_size_online = Scenario.high[18]
    
    # ---------------------------------------------Retail - Warehouse ---------------------------------------------#
    global Retail_energy_use
    Retail_energy_use = [] # MJ/item
    global R_count
    R_count = 0
    if Scenario.Select_Level[19] == 'low':
        Retail_energy_use = Scenario.low[19]
    elif Scenario.Select_Level[19] == 'med':
        Retail_energy_use = Scenario.med[19]
        R_count = 1
    else: 
        Retail_energy_use = Scenario.high[19]
        R_count = 2
    
    global Retail_Warehouse_energy_use
    Retail_Warehouse_energy_use = [] # MJ/item
    global O_count 
    O_count = 0
    if Scenario.Select_Level[20] == 'low':
        Retail_Warehouse_energy_use = Scenario.low[20]
    elif Scenario.Select_Level[20] == 'med':
        Retail_Warehouse_energy_use = Scenario.med[20]
        O_count = 1
    else: 
        Retail_Warehouse_energy_use = Scenario.high[20]
        O_count = 2
    
    global Etail_Warehouse_energy_use
    Etail_Warehouse_energy_use = [] # MJ/item
    global A_count
    A_count = 0
    if Scenario.Select_Level[21] == 'low':
        Etail_Warehouse_energy_use = Scenario.low[21]
    elif Scenario.Select_Level[21] == 'med':
        Etail_Warehouse_energy_use = Scenario.med[21]
        A_count = 1
    else: 
        Etail_Warehouse_energy_use = Scenario.high[21]
        A_count = 2
        
    return

# --------------------------------------------- OmniChannel----------------------------------------------------#
def Retail_Online_perc(Year_selected,Market_Population,option):
    if option == "1": # Retail
        Retail_per = 1
    elif option == "2" or option == "3": # E-tail
        Retail_per = 0
    elif option == "4": # Both
        Retail_per = (Market_Population['In-store'][Year_selected]+
                      Market_Population['Both'][Year_selected])/(Market_Population['In-store'][Year_selected]+
                                                                 Market_Population['Online'][Year_selected]+
                                                                 Market_Population['Both'][Year_selected])

    return(Retail_per)

# ------------------------------------------------------------------------------------------------------------ #

## 1.6. Read the VMT + Emissions from the shopping travel model

Read the file "Tables.xls"  with the aggregate simulation results

In [ ]:
def VMT_3(MSA_op):    
    Market_Population = MSA_Market_Population(str(MSA_op))
    # File name
    MSA = str(MSA_op)
    if MSA == "1":
        name = 'CHI'
    elif MSA == "2":
        name = 'DC'
    elif MSA == "3":
        name = 'LA'
    elif MSA == "4":
        name = 'NYC'
    elif MSA == "5":
        name = 'SF'
        
    # Get the market population
    if (str(Market_Population.loc[Years[0],'Market']) == 'H'):
        sM = 'high'
    else:
        sM = 'mod'
    if (str(Market_Population.loc[Years[0],'Population']) == 'H'):
        sP = 'high'
    else:
        sP = 'mod'
    
    global Shopping_Trip_Distance
    Shopping_Trip_Distance = pd.DataFrame({'Years': Years.copy()})
    Shopping_Trip_Distance['VMT'] = np.zeros(4)
    Shopping_Trip_Distance.set_index('Years', inplace=True)
    
    global Distance_Delivery_Route
    Distance_Delivery_Route = pd.DataFrame({'Years': Years.copy()})
    Distance_Delivery_Route['VMT'] = np.zeros(4)
    Distance_Delivery_Route.set_index('Years', inplace=True)
    
    global Total_VMT
    Total_VMT = []
    
    global Number_Items
    Number_Items = []
    
    global CO2_lastmile
    CO2_lastmile = []

    global Total_CO2_file
    Total_CO2_file = []
    
    global NOX_lastmile
    OX_lastmile = []

    global PM25_lastmile
    PM25_lastmile = []

    global SO2_lastmile
    SO2_lastmile =[]

    # Call scenario
    Scenario = Purchasing_Scenarios(str(option)) 
    Scenario_Level(Scenario)
    
    # Initialization
    Total_distance_save = []
    CO2_LM2 = []
    NOX_LM2 = []
    PM25_LM2 = []
    SO2_LM2 = []
    Total_CO2_aux = []
    Total_CO2_save = []

    for i in Years:
        # Print year
        print(i)     
        
        # Read files
        file_name = Path2 + '/{}/predicted_pop_{}_{}_{}_{}.csv'.format(name,name,i,sM,sP)
        MC = pd.read_csv(file_name)

        # VMT per tour
        Distance = MC.groupby(['DEPVAR_2'])[['VMT']].mean()
        
        # CO2 emissions
        CO2_LM1 = MC.groupby(['DEPVAR_2'])[['CO2']].mean()

        # Air pollutants
        NOX_LM1 = MC.groupby(['DEPVAR_2'])[['NOx']].mean()
        PM25_LM1 = MC.groupby(['DEPVAR_2'])[['PM25']].mean()
        SO2_LM1 = MC.groupby(['DEPVAR_2'])[['SO2']].mean()

        # Stops per shopping tour
        Stops = MC.groupby(['DEPVAR_2'])[['P2']].mean()
        
        # For last mile delivery
        Distance_Delivery_Route['VMT'][i] = Distance['VMT'][3]
        
        # For Customer transport
        Shopping_Trip_Distance['VMT'][i] = Distance['VMT'][2]/Stops['P2'][2]
                      
        # ------------------------------------------- Total VMT per year ------------------------------------------------ #
        # Copy to avoid warnings
        MC_copy =  pd.DataFrame(MC.copy()) 
       
        # "selecting" rows where case numbers are non 1 (no travel)
        MC_copy2 = MC_copy.loc[MC_copy['DEPVAR_2'] != 1] 
        
        # Number of Items
        if str(option) == '1':
            IS = len(MC_copy2.loc[MC_copy2['DEPVAR_2'] == 2]) * Basket_Size_InStore
            Number_Items.append(IS)
        elif (str(option) == '2' or str(option) == '3'):
            IO = len(MC_copy2.loc[MC_copy2['DEPVAR_2'] == 3]) * Basket_size_online
            Number_Items.append(IO)
        elif str(option) == '4':
            O = len(MC_copy2.loc[MC_copy2['DEPVAR_2'] == 4]) * Basket_size_online
            Number_Items.append(O)
            
        # Making online stops 1, to avoid online divide by stops
        MC_copy2.loc[(MC_copy2['DEPVAR_2'] == 3),'P2'] = 1 
        MC_copy2.loc[(MC_copy2['DEPVAR_2'] == 4),'P2'] = 1 # Making online+in-store stops 1 for avoiding online divide by stops
        
        # Distance/stops
        MC_copy2.loc[:,'VMT_P2'] = MC_copy2.loc[:,'VMT']/MC_copy2.loc[:,'P2'] 
        
        # Total distance
        Total_distance = MC_copy2.groupby(['DEPVAR_2'])[['VMT_P2']].sum() 
        Total_distance_save.append(np.transpose(Total_distance['VMT_P2'])) # Saving distance

        # Save emissions per year
        CO2_LM2.append(np.transpose(CO2_LM1))
        NOX_LM2.append(np.transpose(NOX_LM1))
        PM25_LM2.append(np.transpose(PM25_LM1))
        SO2_LM2.append(np.transpose(SO2_LM1))

        # Total emissions
        Total_CO2_aux = MC_copy2.groupby(['DEPVAR_2'])[['CO2']].sum() 
        Total_CO2_save.append(np.transpose(Total_CO2_aux['CO2'])) # Saving CO2
        
        # Delet not needed variables
        del CO2_LM1
        del NOX_LM1
        del PM25_LM1
        del SO2_LM1
        del MC_copy
        del MC_copy2
        del MC
        del file_name
        del Total_CO2_aux
        
    # Save Total VMT 
    Total_VMT = pd.DataFrame(np.transpose(Total_distance_save), columns=Years)
    print("Total VMT")
    display(Total_VMT)

    # Total CO2
    Total_CO2_file = pd.DataFrame(np.transpose(Total_CO2_save), columns=Years)
    print("Total CO2")
    display(Total_CO2_file)
    
    # Save total emissions
    def df(df): # To save as dataframe
        L = np.transpose(df)
        ones = np.ones((2,4))
        M = pd.DataFrame(ones, columns = ['2020', '2030', '2040', '2050'])
        M.iloc[0,:] = pd.DataFrame(L[1])
        M.iloc[1,:] = pd.DataFrame(L[2])
        return(M)
    
    CO2_lastmile = df(CO2_LM2) #gCO2
    NOX_lastmile = df(NOX_LM2)
    PM25_lastmile = df(PM25_LM2)
    SO2_lastmile = df(SO2_LM2)
    
    del Total_distance_save
    del Total_CO2_save
    del CO2_LM2
    del NOX_LM2
    del PM25_LM2
    del SO2_LM2
    
    return
    
# Call function
VMT_3(MSA_op)


2020


C:\Users\cata1\AppData\Local\Temp\ipykernel_45248\2827758225.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MC_copy2.loc[:,'VMT_P2'] = MC_copy2.loc[:,'VMT']/MC_copy2.loc[:,'P2']


2030


C:\Users\cata1\AppData\Local\Temp\ipykernel_45248\2827758225.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MC_copy2.loc[:,'VMT_P2'] = MC_copy2.loc[:,'VMT']/MC_copy2.loc[:,'P2']


2040


## 2. Wherehouse characteristics


### 2.1. Wherehouse Efficient E-tail 

This section collects assumptions and other parameters needed to estimate the effects of pushing products through the E-tail distribution network.

Reference to EGRID:
60 lbs/MWh,min state from US EPA EGRID Database, 2016. https://www.epa.gov/energy/emissions-generation-resource-integrated-database-egrid 

In [ ]:
def WarehouseEtail(Year_selected):
    # -------------------------------------Annual Values ---------------------------------------------------- #
    #  2012 CBECS Data; https://www.eia.gov/consumption/commercial/data/2012/c&e/cfm/c30.php. Low average and high values for the  regions in the US
    Nat_Gas_Use_cuft = np.array([7, 12, 27]) # cu ft/sq ft for low, med, high
    Nat_Gas_Use_BTU = Nat_Gas_Use_cuft * 1030 # BTU/sq ft low, med, high
    Nat_Gas_Use_Joule = Nat_Gas_Use_BTU * 1055 # Joule/sq ft low, med, high
    Nat_Gas_Use_MJ = Nat_Gas_Use_Joule / 1000000 # MJ/sq ft low, med, high
    
    # Similar to above, CBECS Data; https://www.eia.gov/consumption/commercial/data/2012/c&e/cfm/c20.php
    Electricity_Use_kWh = np.array([5, 8.5, 12]) # kWh/sq ft for low, med, high
    Electricity_Use_MJe = Electricity_Use_kWh * 3.6 # MJe/sf for low, med, high
    
    # ---------------------------------- Estimates 2018------------------------------------------------------- #
    GLOBAL_items_net_of_returns = 12       # Billion items
    Value_per_item = 23.08                 # per item
    Items_per_package = 1.2                # items/package
    GLOBAL_value_per_package = 27.70       # per package
    GLOBAL_net_packages_after_returns = 10 # Billion packages
    Estimated_return_rate = 0.2            # 20%       
    Returns_at_20_total_below = 2.5        # Billion packages
    TOTAL_GROSS_OUTBOUND_PACKAGES = 12.5  # Billion packages
    US_PERCENT_OF_THE_ABOVE = 0.65        # 65%
    US_TOTAL_GROSS_OUTBOUND_PACKAGES = 8.125 # Billion packages
    US_Returned_packages = 1.625           # Billion packages
    US_Net_outbound_packages= 6.5          # Billion packages
    Estimated_US_Net_outbound_items = 7.8  # Billion items
    Assumed_items_per_return_package = 1   # items/package
    Estimated_US_returned_items = 1.625    # Billion items
    Estimated_US_Net_outbound_items = 9.425 # billion items
    
    # ---------------------------- Estimated flows for Ecommerce ---------------------------------------------- #
    Type_Number = np.array([162, 53, 10, 47, 152, 5])
    Avg_Size = np.array([750000, 40000, 600000, 350000, 90000, 150000]) #sq ft
    
    # Range of Annual Shipments "flow through of total" per facility (take all the ~9 billion and assume they all go through this facility type, divide by number of facilities in the type)
    global E_Summary_range_used
    E_Summary_range_used = np.array([[0.9, 0.3, 0.3, 0.6, 0.6, Energy_Intensity_Air_perc],[1, 0.4, 0.5, 0.8, 0.8, Energy_Intensity_Air_perc]])
    low = np.array(E_Summary_range_used [0,]*Estimated_US_Net_outbound_items*(10**9)/Type_Number) # (% of tot ship flow thru)
    med = np.array((E_Summary_range_used [0,]+E_Summary_range_used [1,])/2*Estimated_US_Net_outbound_items*(10**9)/Type_Number) # (% of tot ship flow thru)
    high = np.array(E_Summary_range_used [1,]*Estimated_US_Net_outbound_items*(10**9)/Type_Number) # (% of tot ship flow thru)

    # Emission Rates (eGRID2021)
    ERates = [60/2.2/1000, 940/2.2/1000, 2000/2.2/1000]
    #ERates = [479/2.2/1000, 531.7/2.2/1000, 1047/2.2/1000] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    NOxRates = [0.000181818,0.000227273,0.000363636] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    PMRates = [7.09091E-06,1.42727E-05,0.000420773] # https://www.epa.gov/sites/default/files/2020-07/documents/draft_egrid_pm_white_paper_7-20-20.pdf
    SO2Rates = [6.81818E-06,1.72727E-05,2.59091E-05] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf

    class Fulfillment():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[0])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[0]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[0]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_Item_Shipped = np.array([Energy_Use[0]/high[0],Energy_Use[1]/med[0],Energy_Use[2]/low[0]]) # (MJ/item)
        Carbon_emissions_Per_Item_Shipped = np.array([Carbon_Use[0]/high[0],Carbon_Use[1]/med[0],Carbon_Use[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0],Carbon_UseNOx[1]/med[0],Carbon_UseNOx[2]/low[0]]) #(kg NOx/item) - for g NOx, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = np.array([Carbon_UsePM[0]/high[0],Carbon_UsePM[1]/med[0],Carbon_UsePM[2]/low[0]]) #(kg PM/item) - for g PM, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0],Carbon_UseSO2[1]/med[0],Carbon_UseSO2[2]/low[0]]) #(kg SO2/item) - for g SO2, multiply by 1000!
       
    class Prime_Now_Hubs():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[1])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[1]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[1]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_Item_Shipped = np.array([Energy_Use[0]/high[1],Energy_Use[1]/med[1],Energy_Use[2]/low[1]]) # (MJ/item)
        Carbon_emissions_Per_Item_Shipped = np.array([Carbon_Use[0]/high[1],Carbon_Use[1]/med[1],Carbon_Use[2]/low[1]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0],Carbon_UseNOx[1]/med[0],Carbon_UseNOx[2]/low[0]]) #(kg NOx/item) - for g NOx, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = np.array([Carbon_UsePM[0]/high[0],Carbon_UsePM[1]/med[0],Carbon_UsePM[2]/low[0]]) #(kg PM/item) - for g PM, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0],Carbon_UseSO2[1]/med[0],Carbon_UseSO2[2]/low[0]]) #(kg SO2/item) - for g SO2, multiply by 1000!

    class Inbound_Sortation():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[2])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[2]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[2]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_Item_Shipped = np.array([Energy_Use[0]/high[2],Energy_Use[1]/med[2],Energy_Use[2]/low[2]]) # (MJ/item)
        Carbon_emissions_Per_Item_Shipped = np.array([Carbon_Use[0]/high[2],Carbon_Use[1]/med[2],Carbon_Use[2]/low[2]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0],Carbon_UseNOx[1]/med[0],Carbon_UseNOx[2]/low[0]]) #(kg NOx/item) - for g NOx, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = np.array([Carbon_UsePM[0]/high[0],Carbon_UsePM[1]/med[0],Carbon_UsePM[2]/low[0]]) #(kg PM/item) - for g PM, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0],Carbon_UseSO2[1]/med[0],Carbon_UseSO2[2]/low[0]]) #(kg SO2/item) - for g SO2, multiply by 1000!

    class Outbound_Sortation():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[3])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[3]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[3]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_Item_Shipped = np.array([Energy_Use[0]/high[3],Energy_Use[1]/med[3],Energy_Use[2]/low[3]]) # (MJ/item)
        Carbon_emissions_Per_Item_Shipped = np.array([Carbon_Use[0]/high[3],Carbon_Use[1]/med[3],Carbon_Use[2]/low[3]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0],Carbon_UseNOx[1]/med[0],Carbon_UseNOx[2]/low[0]]) #(kg NOx/item) - for g NOx, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = np.array([Carbon_UsePM[0]/high[0],Carbon_UsePM[1]/med[0],Carbon_UsePM[2]/low[0]]) #(kg PM/item) - for g PM, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0],Carbon_UseSO2[1]/med[0],Carbon_UseSO2[2]/low[0]]) #(kg SO2/item) - for g SO2, multiply by 1000!

    class Delivery_Stations():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[4])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[4]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[4]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_Item_Shipped = np.array([Energy_Use[0]/high[4],Energy_Use[1]/med[4],Energy_Use[2]/low[4]]) # (MJ/item)
        Carbon_emissions_Per_Item_Shipped = np.array([Carbon_Use[0]/high[4],Carbon_Use[1]/med[4],Carbon_Use[2]/low[4]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0],Carbon_UseNOx[1]/med[0],Carbon_UseNOx[2]/low[0]]) #(kg NOx/item) - for g NOx, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = np.array([Carbon_UsePM[0]/high[0],Carbon_UsePM[1]/med[0],Carbon_UsePM[2]/low[0]]) #(kg PM/item) - for g PM, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0],Carbon_UseSO2[1]/med[0],Carbon_UseSO2[2]/low[0]]) #(kg SO2/item) - for g SO2, multiply by 1000!

    class Airport_Hubs():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[5])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[5]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[5]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_Item_Shipped = np.array([Energy_Use[2]/high[5],Energy_Use[2]/med[5],Energy_Use[2]/low[5]]) # (MJ/item)
        Carbon_emissions_Per_Item_Shipped = np.array([Carbon_Use[2]/high[5],Carbon_Use[2]/med[5],Carbon_Use[2]/low[5]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0],Carbon_UseNOx[1]/med[0],Carbon_UseNOx[2]/low[0]]) #(kg NOx/item) - for g NOx, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = np.array([Carbon_UsePM[0]/high[0],Carbon_UsePM[1]/med[0],Carbon_UsePM[2]/low[0]]) #(kg PM/item) - for g PM, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0],Carbon_UseSO2[1]/med[0],Carbon_UseSO2[2]/low[0]]) #(kg SO2/item) - for g SO2, multiply by 1000!

    # ------------------------------------------- Outputs ----------------------------------------------------- #
    A_Type_Center = [Fulfillment, Prime_Now_Hubs, Inbound_Sortation, 
                   Outbound_Sortation, Delivery_Stations, Airport_Hubs]
    Total_Energy = np.array([0, 0, 0])
    Total_Carbon = np.array([0, 0, 0])
    global Total_CarbonNOx
    Total_CarbonNOx = np.array([0, 0, 0])
    global Total_CarbonPM
    Total_CarbonPM = np.array([0, 0, 0])
    global Total_CarbonSO2
    Total_CarbonSO2 = np.array([0, 0, 0])
    for i in A_Type_Center:
        Total_Energy = i.Energy_Use_Per_Item_Shipped + Total_Energy
        Total_Carbon = i.Carbon_emissions_Per_Item_Shipped + Total_Carbon
        Total_CarbonNOx = i.Carbon_emissions_Per_Item_ShippedNOx + Total_CarbonNOx
        Total_CarbonPM = i.Carbon_emissions_Per_Item_ShippedPM + Total_CarbonPM
        Total_CarbonSO2 = i.Carbon_emissions_Per_Item_ShippedSO2 + Total_CarbonSO2
        
    return(Total_Carbon)
    # ---------------------------------------------------------------------------------------------------------- #

WarehouseEtail(2020)

In [ ]:
Total_CarbonNOx

### 2.2. Warehouse Retail (Omni)
This section collects assumptions and other parameters needed to estimate the effects associated with the effects of pushing products through a retail distribution network.This is thus representative of a large-scale retailer and not a smaller, niche, retailer.


In [ ]:
def WarehouseRetail(Year_selected):
    # -------------------------------------Annual Values ---------------------------------------------------- #
    #  2012 CBECS Data; https://www.eia.gov/consumption/commercial/data/2012/c&e/cfm/c30.php. Low average and high values for the  regions in the US
    Nat_Gas_Use_cuft = np.array([7, 12, 27]) # cu ft/sq ft for low, med, high
    Nat_Gas_Use_BTU = Nat_Gas_Use_cuft * 1030 # BTU/sq ft low, med, high
    Nat_Gas_Use_Joule = Nat_Gas_Use_BTU * 1055 # Joule/sq ft low, med, high
    Nat_Gas_Use_MJ = Nat_Gas_Use_Joule / 1000000 # MJ/sq ft low, med, high
    
    # Similar to above, CBECS Data; https://www.eia.gov/consumption/commercial/data/2012/c&e/cfm/c20.php
    Electricity_Use_kWh = np.array([5, 8.5, 12]) # kWh/sq ft for low, med, high
    Electricity_Use_MJe = Electricity_Use_kWh * 3.6 # MJe/sf for low, med, high
    
    # ---------------------------------- Total Estimates 2018------------------------------------------------ #
    Total_Annual_Target_Revenue = 74 # Billion items
    Avg_Price_per_item = 23 # $ per item
    Tot_items_sold = Total_Annual_Target_Revenue/Avg_Price_per_item # Billions
    Avg_items_bulk_carton_shipped = 24 # per package
    Total_Annual_Target_Cartons_Shipped = Tot_items_sold/Avg_items_bulk_carton_shipped # Billion packages
        
    # ---------------------------- Estimated flows for Ecommerce -------------------------------------------- #
    
    Type_Number = np.array([26, 4, 5, 5])
    Avg_Size = np.array([1500000, 1800000, 0, 700000]) #sq ft
    
    # Range of Annual Shipments "flow through of total" per facility (take all the ~9 billion and assume they all go through this facility type, divide by number of facilities in the type)
    global R_Summary_range_used
    R_Summary_range_used = np.array([[0.9, 0.1, 0.1, 0.05],[1, 0.2, 0.3, 0.1]])
    low = np.array(R_Summary_range_used [0,]*Total_Annual_Target_Cartons_Shipped*(10**9)/Type_Number) # (% of tot ship flow thru)
    med = np.array(((R_Summary_range_used [0,]+R_Summary_range_used [1,])/2)*Total_Annual_Target_Cartons_Shipped*(10**9)/Type_Number) # (% of tot ship flow thru)
    high = np.array(R_Summary_range_used [1,]*Total_Annual_Target_Cartons_Shipped*(10**9)/Type_Number) # (% of tot ship flow thru)

    # Emission Rates (eGRID2021)
    ERates = [60/2.2/1000, 940/2.2/1000, 2000/2.2/1000]
    #ERates = [479/2.2/1000, 531.7/2.2/1000, 1047/2.2/1000] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    NOxRates = [0.000181818,0.000227273,0.000363636] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    PMRates = [7.09091E-06,1.42727E-05,0.000420773] # https://www.epa.gov/sites/default/files/2020-07/documents/draft_egrid_pm_white_paper_7-20-20.pdf
    SO2Rates = [6.81818E-06,1.72727E-05,2.59091E-05] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    
    class Reg_Dist_Centers():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[0])
        E_Electricity_Use_Anual_FacilitykWh = np.array(Electricity_Use_kWh * Avg_Size[0])                                                                       
        E_Electricity_Use_Anual_FacilityMJ = np.array(Electricity_Use_MJe * Avg_Size[0])
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_CARTON_Shipped = np.array([Energy_Use[0]/high[0], Energy_Use[1]/med[0], Energy_Use[2]/low[0]]) # (MJ/item)
        Energy_Use_Per_Item_Shipped = Energy_Use_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped # (MJ/item)
        Carbon_emissions_Per_CARTON_Shipped = np.array([Carbon_Use[0]/high[0], Carbon_Use[1]/med[0], Carbon_Use[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_Shipped = Carbon_emissions_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0], Carbon_UseNOx[1]/med[0], Carbon_UseNOx[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = Carbon_emissions_Per_CARTON_ShippedNOx/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedPM = np.array([Carbon_UsePM[0]/high[0], Carbon_UsePM[1]/med[0], Carbon_UsePM[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = Carbon_emissions_Per_CARTON_ShippedPM/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0], Carbon_UseSO2[1]/med[0], Carbon_UseSO2[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = Carbon_emissions_Per_CARTON_ShippedSO2/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!

    class Import_Dist_Centers():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[1])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[1]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[1]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_CARTON_Shipped = np.array([Energy_Use[0]/high[1], Energy_Use[1]/med[1], Energy_Use[2]/low[1]]) # (MJ/item)
        Energy_Use_Per_Item_Shipped = Energy_Use_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped # (MJ/item)
        Carbon_emissions_Per_CARTON_Shipped = np.array([Carbon_Use[0]/high[1], Carbon_Use[1]/med[1], Carbon_Use[2]/low[1]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_Shipped = Carbon_emissions_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0], Carbon_UseNOx[1]/med[0], Carbon_UseNOx[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = Carbon_emissions_Per_CARTON_ShippedNOx/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedPM = np.array([Carbon_UsePM[0]/high[0], Carbon_UsePM[1]/med[0], Carbon_UsePM[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = Carbon_emissions_Per_CARTON_ShippedPM/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0], Carbon_UseSO2[1]/med[0], Carbon_UseSO2[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = Carbon_emissions_Per_CARTON_ShippedSO2/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!

    class Food_Dist_Centers():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[2])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[2]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[2]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_CARTON_Shipped = np.array([Energy_Use[0]/high[2], Energy_Use[1]/med[2], Energy_Use[2]/low[2]]) # (MJ/item)
        Energy_Use_Per_Item_Shipped = Energy_Use_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped # (MJ/item)
        Carbon_emissions_Per_CARTON_Shipped = np.array([Carbon_Use[0]/high[2], Carbon_Use[1]/med[2], Carbon_Use[2]/low[2]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_Shipped = Carbon_emissions_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0], Carbon_UseNOx[1]/med[0], Carbon_UseNOx[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = Carbon_emissions_Per_CARTON_ShippedNOx/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedPM = np.array([Carbon_UsePM[0]/high[0], Carbon_UsePM[1]/med[0], Carbon_UsePM[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = Carbon_emissions_Per_CARTON_ShippedPM/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0], Carbon_UseSO2[1]/med[0], Carbon_UseSO2[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = Carbon_emissions_Per_CARTON_ShippedSO2/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!

    class E_commerce_DC():
        E_Nat_Gas_Use_Ave_Anual_FacilityMJ = np.array(Nat_Gas_Use_MJ * Avg_Size[3])
        E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Avg_Size[3]                                                                       
        E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Avg_Size[3]
        Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
        C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
        C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * ERates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
        C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
        C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
        C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
        C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
        Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
        Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
        Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
        Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
        Energy_Use_Per_CARTON_Shipped = np.array([Energy_Use[0]/high[3], Energy_Use[1]/med[3], Energy_Use[2]/low[3]]) # (MJ/item)
        Energy_Use_Per_Item_Shipped = Energy_Use_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped # (MJ/item)
        Carbon_emissions_Per_CARTON_Shipped = np.array([Carbon_Use[0]/high[3], Carbon_Use[1]/med[3], Carbon_Use[2]/low[3]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_Shipped = Carbon_emissions_Per_CARTON_Shipped/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedNOx = np.array([Carbon_UseNOx[0]/high[0], Carbon_UseNOx[1]/med[0], Carbon_UseNOx[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedNOx = Carbon_emissions_Per_CARTON_ShippedNOx/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedPM = np.array([Carbon_UsePM[0]/high[0], Carbon_UsePM[1]/med[0], Carbon_UsePM[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedPM = Carbon_emissions_Per_CARTON_ShippedPM/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_CARTON_ShippedSO2 = np.array([Carbon_UseSO2[0]/high[0], Carbon_UseSO2[1]/med[0], Carbon_UseSO2[2]/low[0]]) #(kg CO2/item) - for g CO2, multiply by 1000!
        Carbon_emissions_Per_Item_ShippedSO2 = Carbon_emissions_Per_CARTON_ShippedSO2/Avg_items_bulk_carton_shipped  #(kg CO2/item) - for g CO2, multiply by 1000!

    Type_Center = [Reg_Dist_Centers, Import_Dist_Centers, Food_Dist_Centers, E_commerce_DC]
    
    Total_Energy_retail_ecomm = np.array([0, 0, 0])
    Total_Carbon_retail_ecomm = np.array([0, 0, 0])
    Total_NOx_retail_ecomm = np.array([0, 0, 0])
    Total_PM_retail_ecomm = np.array([0, 0, 0])
    Total_SO2_retail_ecomm = np.array([0, 0, 0])
    for i in Type_Center:
        Total_Energy_retail_ecomm = i.Energy_Use_Per_Item_Shipped + Total_Energy_retail_ecomm
        Total_Carbon_retail_ecomm = i.Carbon_emissions_Per_Item_Shipped + Total_Carbon_retail_ecomm
        Total_NOx_retail_ecomm = i.Carbon_emissions_Per_Item_ShippedNOx + Total_NOx_retail_ecomm
        Total_PM_retail_ecomm = i.Carbon_emissions_Per_Item_ShippedPM + Total_PM_retail_ecomm
        Total_SO2_retail_ecomm = i.Carbon_emissions_Per_Item_ShippedSO2 + Total_SO2_retail_ecomm
    
    Total_Energy_retail_only = (Total_Energy_retail_ecomm - E_commerce_DC.Energy_Use_Per_Item_Shipped)
    Total_Carbon_retail_only = (Total_Carbon_retail_ecomm - E_commerce_DC.Carbon_emissions_Per_Item_Shipped)
    Total_NOx_retail_only = (Total_NOx_retail_ecomm - E_commerce_DC.Carbon_emissions_Per_Item_ShippedNOx)
    Total_PM_retail_only = (Total_PM_retail_ecomm - E_commerce_DC.Carbon_emissions_Per_Item_ShippedPM)
    Total_SO2_retail_only = (Total_SO2_retail_ecomm - E_commerce_DC.Carbon_emissions_Per_Item_ShippedSO2)
    
    return(Total_Carbon_retail_ecomm,Total_Carbon_retail_only,Total_NOx_retail_ecomm,Total_NOx_retail_only,
           Total_PM_retail_ecomm,Total_PM_retail_only,Total_SO2_retail_ecomm,Total_SO2_retail_only)
    # ---------------------------------------------------------------------------------------------------------- #   


## **3. Model Parameter** 

### 3.1. Product information

Total weight is getting from the user info and totalized by the summ of average weigth per item and weight of the shipping package

In [ ]:
def ItemShipment():
    
    # ------------------------------------------- Outputs --------------------------------------------------- #
    # Total Weight of each item
    Total_Item_Weight = Average_Weigth_Item + Weight_shipping_package
    
    
    return(Total_Item_Weight)
    # ------------------------------------------------------------------------------------------------------- #

### 3.2. Packaging

Average item volume is an assumption that drives the size of packaging needed (all assumed to be corrugated cardboard), and packaging use is linearly proportional to energy and climate effects.  

References: 
U.S. EPA’s Waste Action and Reduction Model (EPA 2008)

In [ ]:
def Packaging(Year_selected):
    # -------------------------------- Bulk packaging used for products ----------------------------------------
    # i.e., the large boxes that would come from a factory to a retailer or to an e-commerce fulfillment center)
    # From ASU Paper
    Volume_36_36_24 = 36*36*24     # in3
    Surface_area = 4*36*24+2*36*36 # in2

    # Number of items in bulk packaging
    N_items_bulk = Volume_36_36_24 / Average_item_volume

    # Per item in bulk packaging
    Per_item_bulk = Surface_area / N_items_bulk # in2/item

    # Per item in Flex packaging 
    # Flex Packaging assumed from the ratio of product dimensions of 2x10x8 and packaging at 3x12x10, for ratio of 2.33
    Per_item_flex = Per_item_bulk * 1.6 # in2/item


    # ------------------- The energy and carbon effects US EPA's WARM Model (2008)-------------------------------
    # En/CO2e of cardboard
    Carbon_Carboard_NoForest = 0.2 #ton CO2e/ton cardboard, does not include forest sequestration
    Carbon_Carboard_Forest_Aver = 1.5 #ton CO2e/ton cardboard, does include forest, displace average
    Carbon_Carboard_Forest_Virg = 2.2 #ton CO2e/ton cardboard, does include forest, displace virgin
    
    # Energy of cardboard
    Energy_Carboard_Virg_MBTU = 25 + 1.3 + 0.3 #Energy result, virgin
    Energy_Carboard_Recy_MBTU = 11.7 + 0.8 + 0.3 #Energy result, recycled
    Energy_Carboard_Virg_MJ = Energy_Carboard_Virg_MBTU * 1055 #Energy result, virgin
    Energy_Carboard_Recy_MJ = Energy_Carboard_Recy_MBTU * 1055 #Energy result, recycled
    Assumed_recylcled = 0.35

    Total_Energy_Carboard_MBTU = (1-Assumed_recylcled)*Energy_Carboard_Virg_MBTU+Assumed_recylcled*Energy_Carboard_Recy_MBTU
    Total_Energy_Carboard_MJ = Total_Energy_Carboard_MBTU * 1055


    # ------------------------------- Density of cardboard-------------------------------------------------------
    # uri.com
    Lb_package_1 = 4.25
    Lb_package_2 = 7.68
    Lb_package_3 = 13.4
    Surface_package_1 = 2 * 17 * 17 + 4 * 17 * 30 #in2
    Surface_package_2 = 2 * 22 * 22 + 4 * 22 * 36 #in2
    Surface_package_3 = 6 * 36 * 36               #in2
    Density_1 = Lb_package_1 / Surface_package_1 #lb/in2
    Density_2 = Lb_package_2 / Surface_package_2 #lb/in2
    Density_3 = Lb_package_3 / Surface_package_3 #lb/in2
    
    Density_Aver_lb = (Density_1 + Density_2 + Density_3)/3 #lb/in2
    Density_Aver_kg = Density_Aver_lb * 0.4536 #kg cardboard/in2
    
    # ------------------------------- CO2 intensity of cardboard ------------------------------------------------
    # low, med, high
    CO2_intensity_kg = [Carbon_Carboard_NoForest * Density_Aver_kg, Carbon_Carboard_Forest_Aver * Density_Aver_kg,
                        Carbon_Carboard_Forest_Virg * Density_Aver_kg] # kg CO2/in2
    
    # low, med, high
    Energy_intensity_lb = [Density_Aver_kg * Energy_Carboard_Recy_MJ / 1000, 
                           Density_Aver_kg * Total_Energy_Carboard_MJ / 1000,
                           Density_Aver_kg * Energy_Carboard_Virg_MJ / 1000] #MJ/in2
    
    # --------------------------------------- Outputs ----------------------------------------------------------- #
    # -------------------------------- Packaging per Item ------------------------------------------------------- #
    Packaging_item_gCO2_Retail = np.array([CO2_intensity_kg[0]*Per_item_bulk*1000, CO2_intensity_kg[1]*Per_item_bulk*1000,
                               CO2_intensity_kg[2]*Per_item_bulk*1000], float)*CO2_Paper[0][str(Year_selected)]
    Packaging_item_gCO2_Etail = np.array([CO2_intensity_kg[0]*Per_item_flex*1000, CO2_intensity_kg[1]*Per_item_flex*1000,
                               CO2_intensity_kg[2]*Per_item_flex*1000], float)*CO2_Paper[0][str(Year_selected)]
    Packaging_item_MJ_Retail = np.array([Energy_intensity_lb[0]*Per_item_bulk,Energy_intensity_lb[1]*Per_item_bulk,
                                Energy_intensity_lb[2]*Per_item_bulk], float)*MJ_Paper[0][str(Year_selected)]
    Packaging_item_MJ_Etail = np.array([Energy_intensity_lb[0]*Per_item_flex,Energy_intensity_lb[1]*Per_item_flex,
                                Energy_intensity_lb[2]*Per_item_flex], float)*MJ_Paper[0][str(Year_selected)]   
    

    return(Packaging_item_gCO2_Retail, Packaging_item_gCO2_Etail, Packaging_item_MJ_Retail, Packaging_item_MJ_Etail)
    # ---------------------------------------------------------------------------------------------------------- # 
# Pollutants (excel)
Packaging_item_gNOx_Retail = np.array([1.62416E-05,1.98451E-05,3.37522E-05])
Packaging_item_gNOx_Etail = np.array([1.0151E-05,1.24032E-05,2.10951E-05])
Packaging_item_gPM_Retail = np.array([0.043179009,0.052758918,0.089731378])
Packaging_item_gPM_Etail = np.array([0.026986881,0.032974324,0.056082111])
Packaging_item_gSO2_Retail = np.array([1.30725E-06,1.59729E-06,2.71664E-06])
Packaging_item_gSO2_Etail = np.array([8.17034E-07,9.98305E-07,1.6979E-06])


### 3.3. Energy Intensity and Freight

This section includes: 3.3.1. Carbon content from various literature values for energy and carbon contents of fuels and electricity in the US; and 3.3.2. Freight transport where some parameters are collected to estimate the effects associated with the vehicle-level (logistics) effects of pushing products through an e-tail or retail distribution network.


### 3.3.1. Energy Intensity

Reference:
EPA. 2006. Inventory of U.S. Greenhouse Gas Emissions and Sinks, 1990-2004. Washington, D.C.: US Environmental Protection Agency.  


In [ ]:
def CarbonContents(Year_selected): 
    # -------------------------------------- Carbon Content on Fuels -------------------------------------------- #
    # EPA 2006 (below) - note this source is a bit dated but values have not changed significantly since then.  
    # Note, these are the raw values of sources.
    Fuel_Type = ["Finished Motor Gasoline", "Finished Aviation Gasoline", "Kerosene-Type Jet Fuel", 
                 "Distillate Fuel Oil", "Residual Fuel Oil", "Liquefied Refinery Gases", "Natural Gas"]  
    Carbon_Content = np.array([19.34, 18.87, 19.33, 19.95, 21.49, 16.99, 14.47],float) # Tg C/QBTU
    Heat_Content = np.array([5.25, 5.05, 5.67, 5.83, 6.29, 3.85, 1030],float) # MMBTU/bbl or BTU/ft3
    Oxydation_fraction = np.array([0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.995],float) 
    
    # Outputs
    # Converting from barrels to gallons by dividing by 42
    global BTU_gal
    BTU_gal = Heat_Content * 1000000/42 # BTU/gal
    # Converting from Tg C/quadrillion BTU to g CO2/MJ
    global Combustion_Emissions
    Combustion_Emissions = [0] * len(Carbon_Content) # gr CO2/MJ
    for i in range(len(Carbon_Content)): ################################### Include here the carbon from MOVES
        Combustion_Emissions[i] = Carbon_Content[i] * 10**12 * 44 * Oxydation_fraction[i] * 10**6 /(12 * 1055 * 10**15)
    
    # ------------------------------------- Electricity life cycle emissions ----------------------------------- #
    # EPA 2006 (below)
    Source = ["Current Mix", "Low Carbon Sources", "PC Coal w/o CCS", "PC Coal w/ 90% CCS", "IGCC Coal w/o CCS", 
              "IGCC Coal w/ 90% CCS", "Domestic NGCC w/o CCS", "Domestic NGCC w/90% CCS"]
    Upstream = np.array([98.26, 0.0, 106.5, 140.5, 111.5, 129.0, 120.5, 141.5], float)
    global Combustion
    Combustion = np.array([1339.0, 0.0, 1781.0, 234.0, 1866.0, 215.0, 816.0, 96], float)
    
    # Outputs
    gCO2e_kwh = (Upstream + Combustion) * 0.4535 # Total lb CO2/MWh
    global  grCO2e_MJe
    grCO2e_MJe = (gCO2e_kwh / 3.6) * CO2_per_MJ_Electricity[0][str(Year_selected)] # Total g CO2e/kWh
     
    return

# Criteria air pollutants: Finished motor vehicle, Kerosene, Distillate Fuil Oil
Combustion_Pollutants = np.array([[0.033,0.041,0.05,0.032],[0.004,0.005,0.008,0.03],[0.094,0.099,0.026,0.098]])
CFactor = 0.4536/3.6
NOX_Upstream = 0.48
NOX_Combustion = 0.80
grNOX_MJe = (NOX_Upstream + NOX_Combustion) * CFactor # gr NOX/MJe
PM_Upstream = 0.004
PM_Combustion = 0.03
grPM_MJe =(PM_Upstream + PM_Combustion) * CFactor # gr PM25/MJe
SO2_Upstream = 0.055387621
SO2_Combustion = 0.342797103
grSO2_MJe = (SO2_Upstream + SO2_Combustion) * CFactor # gr SO2/MJe

    # ---------------------------------------------------------------------------------------------------------- #

### 3.3.2. Freight

References:
Pahwa, A. and M. Jaller (2020). Evaluating the costs and distribution structure in last mile deliveries under short time-windows. Transportation Research Board Annual Meeting. Washington, D.C.
Location data from http://www.mwpvl.com/html/amazon_com.html


In [ ]:
def Freight(Year_selected):
    Total_Area_of_Continental_US = 3119884 # sq mi
    
    # Functions
    WarehouseEtail(Year_selected)
    WarehouseRetail(Year_selected)
    Total_Item_Weight = ItemShipment()
    
    #--------------------------------------- Summary range used Etail----------------------------------------- #
    F = 1.5 # Factor to increas distance
    global Etail
    class Etail:
        Average_Distance_to_reach = np.array([70.78, 98.12, 284.87, 131.40, 31.68, 900.82])*F
        Etail = np.array([162, 53, 10, 47, 152, 5])
        low = np.array(E_Summary_range_used [0,]*Average_Distance_to_reach) 
        med = np.array((E_Summary_range_used [0,]+E_Summary_range_used [1,])/2*Average_Distance_to_reach) 
        high = np.array(E_Summary_range_used [1,]*Average_Distance_to_reach) 
        Road_Miles_item = np.array([0,0,0])
        for i in range(len(Average_Distance_to_reach)-1):
            Road_Miles_item = np.array([low[i],med[i],high[i]])+Road_Miles_item
        Road_Tone_Mile = Road_Miles_item*Total_Item_Weight/2000*(1+Rate_Online_Purchase)
        Air_Tone_Mile = np.array([low[5],med[5],high[5]])*Total_Item_Weight/2000
    
    # -------------------------------------- Summary range used Retail ---------------------------------------- #
    global Retail
    class Retail:
        Average_Distance_to_reach = np.array([1/26**0.5, 1/4**0.5, 1/5**0.5, 1/5**0.5])*(Total_Area_of_Continental_US**0.5)*0.51*F
        low = np.array(R_Summary_range_used [0,]*Average_Distance_to_reach) 
        med = np.array((R_Summary_range_used [0,]+R_Summary_range_used [1,])/2*Average_Distance_to_reach*1.1) 
        high = np.array(R_Summary_range_used [1,]*Average_Distance_to_reach*1.1) 
        Road_Miles_Ton = np.array([0,0,0])
        for i in range(len(Average_Distance_to_reach)-2):
            Road_Miles_Ton = np.array([low[i],med[i],high[i]])+Road_Miles_Ton
        Road_Tone_Mile = Road_Miles_Ton*Total_Item_Weight/2000
        E_commerce_Road = (np.array([low[3],med[3],high[3]])+Road_Miles_Ton)*Total_Item_Weight/2000

    return
    # ---------------------------------------------------------------------------------------------------------- #


### 3.4. Shipping Option

The model estimates the movement of products throughout the hierarchical structure of typical retailers and e-retailers. There could be a combination of modes (e.g., road and air). As expected, Air freight is more energy and climate intensive than truck-based freight. Moreover, the level of Rush Shipments is an indicator of the intensity of transport required to deliver in very short time windows. Assumed as multiplier based on other ongoing modeling efforts. Now, the rush shipments indicator is de-activated. The impact of rush shipments is introduced by lower last mile efficiency (longer delivery distances per item, to account for the inefficiencies of the first and last leg of the distribution.


In [ ]:
def Distances(Year_selected,Scenario):
    
    # Call functions
    Freight(Year_selected)

    # ------------------------------- Energy per Shipment (MJ) ------------------------------------------ #
    Energy_Intensity = []
    if Type_of_road_vehicle_FTL == "Heavy Duty Truck (25 ton cargo)":
        Energy_Intensity = ((Report_Energy.loc[53,Year_selected]*Report_Distance.loc[53,Year_selected]+
                            Report_Energy.loc[61,Year_selected]*Report_Distance.loc[61,Year_selected]+
                            Report_Energy.loc[62,Year_selected]*Report_Distance.loc[62,Year_selected])/
                            (Report_Distance.loc[53,Year_selected]+Report_Distance.loc[61,Year_selected]+
                             Report_Distance.loc[62,Year_selected]))/25
    elif Type_of_road_vehicle_FTL == "Medium Duty Truck (8 ton cargo)":
        Energy_Intensity = ((Report_Energy.loc[53,Year_selected]*Report_Distance.loc[53,Year_selected]+
                            Report_Energy.loc[52,Year_selected]*Report_Distance.loc[52,Year_selected])/
                            (Report_Distance.loc[53,Year_selected]+Report_Distance.loc[52,Year_selected]))/8
    else:
        Energy_Intensity = Energy_Intensity_Rail*1055/1000000

    # Retail
    global R_Freight_Energy_Road
    R_Freight_Energy_Road =  Retail.Road_Tone_Mile*Energy_Intensity

    # Etail - Road
    global E_Freight_Energy_Road
    E_Freight_Energy_Road = []
    if option == 3:
        E_Freight_Energy_Road = Etail.Road_Tone_Mile * Energy_Intensity
    else: 
        E_Freight_Energy_Road = Retail.E_commerce_Road * Energy_Intensity 

    # Etail - Air
    global E_Freight_Energy_Air
    if Energy_option == "3": #-------------------- For NZE50
        Factor = Etail.Air_Tone_Mile * Energy_Intensity_Air * 1055/1000000
        E_Air = np.array([1*Factor,0.429*Factor,0.577*Factor,0.625*Factor])
        E_Freight_Energy_Air = pd.DataFrame(E_Air, index = ['2020', '2030', '2040', '2050'])
    else:                    #-------------------- Other energy options 
        E_Freight_Energy_Air = Etail.Air_Tone_Mile * Energy_Intensity_Air * 1055/1000000
    
    # Etail - Total 
    E_Freight_Energy_Total = E_Freight_Energy_Road + E_Freight_Energy_Air
    
    # Level of Rush Shipping
    Total_cost = np.array([323,489,812]) # No time windows, >2hr, <2hr
    Multiplier = np.array([Total_cost[1]/Total_cost[0],Total_cost[2]/Total_cost[0]])
    Level_of_Rush_Shipping = np.array([1,Multiplier[0],Multiplier[1]])
    
    return(Level_of_Rush_Shipping)
    # ---------------------------------------------------------------------------------------------------------- #    


### 3.5. Fuel Economy for Last Mile Delivery Vehicles

Similar to the energy intensity section above, three different types of trucks are available in the model, each with a range of fuel economy from transportation literature.  

References:
Jaller, M. and A. Pahwa (2020). Evaluating the Environmental Impacts of Online Shopping: A Behavioral and Transportation Approach. Journal of Transportation Research Part D: Transport and Environment (in print)



In [ ]:
def Enpertmi(Year_selected,Scenario): 
    
    # Call functions
    CarbonContents(Year_selected)
 
    # --------------------------------- Energy intensity of freigth modes ------------------------------------- #
    
    # Mode
    Heavy_Trucks = 20539 # BTU/vehicle-mile, fuel used: 10% Gasoline, 89.6% Diesel, 0.33% LPG
    Heavy_Duty_Truck = 25690 / 25 # BTU/ton-mile, fuel used: 100% Diesel
    Medium_Duty_Truck = 17596 / 8 # BTU/ton-mile, fuel used: 100% Diesel
    Rail = np.array([15152, 337], float) # [BTU/freight car-mile, BTU/ton-mile], fuel used: 100% Diesel
    Certified_Air_Carriers = 3264 # BTU/passenger-mile, fuel used: 100% Jet Fuel
    Air_Carrier_Shipment = (10 * 10**6) * 1.6 / (1055 * 1.1) # BTU/ton-mile, fuel used: 100% Jet Fuel
    


    # -------------------------------------------- Output - SRR------------------------------------------------ #
    if Scenario.Select_Level[14] == 'Light Truck (less than 10,000 lb Truck)':
        Energy_per_mile_per_truck = ((Report_Energy.loc[32,Year_selected]*Report_Distance.loc[32,Year_selected]+
                                      Report_Energy.loc[52,Year_selected]*Report_Distance.loc[52,Year_selected])/
                                     (Report_Distance.loc[32,Year_selected]+Report_Distance.loc[52,Year_selected]))
        
    elif Scenario.Select_Level[14] == 'Medium Truck (20,000 lb)':
        Energy_per_mile_per_truck = ((Report_Energy.loc[53,Year_selected]*Report_Distance.loc[53,Year_selected]+
                                      Report_Energy.loc[52,Year_selected]*Report_Distance.loc[52,Year_selected])/
                                     (Report_Distance.loc[53,Year_selected]+Report_Distance.loc[52,Year_selected]))
        
    else: 
        Energy_per_mile_per_truck =((Report_Energy.loc[53,Year_selected]*Report_Distance.loc[53,Year_selected]+
                                     Report_Energy.loc[61,Year_selected]*Report_Distance.loc[61,Year_selected]+
                                     Report_Energy.loc[62,Year_selected]*Report_Distance.loc[62,Year_selected])/
                                    (Report_Distance.loc[53,Year_selected]+Report_Distance.loc[61,Year_selected]+Report_Distance.loc[62,Year_selected]))

    # -------------------------------------------- Output - Sheet --------------------------------------------- #
    return(Energy_per_mile_per_truck)
    # --------------------------------------------------------------------------------------------------------- #


### 3.6. Computer Usage

The new estimation accounts for data based on the literature about the aggregate energy consumption and the internet infrastructure. The data results in consumption per hour for the transfer of 1 MB of data divided by end user, upstream, and data center.
The analyses also estimate average consumptions for an assumed online purchase that requires accessing a fixed number of website pages per item (e.g., landing, search, item selection, cart, checkout).

Analyses based on:
Costenaro, D., & Duer, A. (2012). The megawatts behind your megabytes: going from data-center to desktop. Proceedings of the 2012 ACEEE Summer Study on Energy Efficiency in Buildings, ACEEE, Washington, 13-65.



In [ ]:
def DataUsage(): 
    #------------------------------------ Total Power GW per Year (2011)----------------------------------------#
    # User
    User_desktop = np.array([28.1, 40.8, 53.4]) # Low, med, high
    User_laptops = np.array([11.3, 13.1, 15.0]) # Low, med, high
    User_smarphones = np.array([0.1, 0.3, 0.5]) # Low, med, high
    USER = User_desktop + User_laptops + User_smarphones
    
    # Trans3
    Trans3_WifiLAN = np.array([1.5, 1.8, 2.0]) # Low, med, high
    Trans3_CellTowers = np.array([1.5, 4.5, 7.5]) # Low, med, high
    Trans3_TelecomSwitches = np.array([0.0, 0.7, 1.4]) # Low, med, high
    Trans3_kmFiberOptics = np.array([0.0, 0.0, 0.0]) # Low, med, high
    Trans3_OpticalRepeters = np.array([1.3, 1.9, 2.4]) # Low, med, high
    Trans3_kmCopper = np.array([0.0, 0.0, 0.0]) # Low, med, high
    Trans3_SignalRepeaters = np.array([1.9, 5.6, 9.3]) # Low, med, high
    Trans3 = Trans3_WifiLAN + Trans3_CellTowers + Trans3_TelecomSwitches + Trans3_kmFiberOptics + Trans3_OpticalRepeters + Trans3_kmCopper + Trans3_SignalRepeaters
    
    # POP Tier 3
    POPT3_Severs = np.array([6.0, 8.7, 11.4]) # Low, med, high
    POPT3_Routers = np.array([1.8, 1.9, 2.0]) # Low, med, high
    POPT3_Cloud = np.array([4.8, 5.4, 6.0]) # Low, med, high
    POPT3_Overhead_power = np.array([12.6, 16.0, 19.4]) # Low, med, high
    POPT3 = POPT3_Severs + POPT3_Routers + POPT3_Cloud + POPT3_Overhead_power
    
    # Trans2
    Trans2_kmFiberOptics = np.array([0.0, 0.0, 0.0]) # Low, med, high
    Trans2_OpticalRepeters = np.array([2.0, 2.8, 3.6]) # Low, med, high
    Trans2 = Trans2_kmFiberOptics + Trans2_OpticalRepeters
    
    # POP Tier 2
    POPT2_Severs = np.array([4.5, 6.5, 8.6]) # Low, med, high
    POPT2_Routers = np.array([1.4, 1.4, 1.5]) # Low, med, high
    POPT2_Cloud = np.array([3.6, 4.1, 4.5]) # Low, med, high
    POPT2_Overhead_power = np.array([7.1, 9.0, 10.9]) # Low, med, high
    POPT2 = POPT2_Severs + POPT2_Routers + POPT2_Cloud + POPT2_Overhead_power
    
    # Trans1
    Trans1_kmFiberOptics = np.array([0.0, 0.0, 0.0]) # Low, med, high
    Trans1_OpticalRepeters = np.array([1.3, 1.9, 2.4]) # Low, med, high
    Trans1 = Trans1_kmFiberOptics + Trans1_OpticalRepeters
    
    # Sub-total
    UPSTREAM = Trans3 + POPT3 + Trans2 + POPT2 + Trans1
    
    # POP Tier 1
    POPT1_Severs = np.array([4.5, 6.5, 8.6]) # Low, med, high
    POPT1_Routers = np.array([1.4, 1.4, 1.5]) # Low, med, high
    POPT1_Cloud = np.array([3.6, 4.1, 4.5]) # Low, med, high
    POPT1_Overhead_power = np.array([2.4, 3.0, 3.6]) # Low, med, high
    DATA_CENTER = POPT1_Severs + POPT1_Routers + POPT1_Cloud + POPT1_Overhead_power
    
    # TOTAL GW per year
    GW_YEAR = USER + UPSTREAM + DATA_CENTER
    
    # ------------------------------------ Average use per GB ---------------------------------------------- #
    # Average use per GB
    Low_percentage = (GW_YEAR[0]-GW_YEAR[1])/GW_YEAR[1] 
    High_percentage = (GW_YEAR[2]-GW_YEAR[1])/GW_YEAR[1] 
    Average_use_per_GB_med = 5.12
    
    # kWh/GB
    Average_use_per_GBkWh = np.array([Average_use_per_GB_med*(1+Low_percentage), Average_use_per_GB_med, Average_use_per_GB_med*(1+High_percentage)])
    Average_use_per_GB_UserkWh = np.array([Average_use_per_GBkWh[0]*USER[0]/GW_YEAR[0],Average_use_per_GBkWh[1]*USER[1]/GW_YEAR[1],Average_use_per_GBkWh[2]*USER[2]/GW_YEAR[2]])
    Average_use_per_GB_UpstreamkWh = np.array([Average_use_per_GBkWh[0]*UPSTREAM[0]/GW_YEAR[0],Average_use_per_GBkWh[1]*UPSTREAM[1]/GW_YEAR[1],Average_use_per_GBkWh[2]*UPSTREAM[2]/GW_YEAR[2]])
    Average_use_per_GB_DatacenterkWh = np.array([Average_use_per_GBkWh[0]*DATA_CENTER[0]/GW_YEAR[0],Average_use_per_GBkWh[1]*DATA_CENTER[1]/GW_YEAR[1],Average_use_per_GBkWh[2]*DATA_CENTER[2]/GW_YEAR[2]])
    
    # MJ/GB
    Average_use_per_GBMJ = Average_use_per_GBkWh*3.6*3
    Average_use_per_GB_UserMJ = Average_use_per_GB_UserkWh*3.6*3
    Average_use_per_GB_UpstreamMJ = Average_use_per_GB_UpstreamkWh*3.6*3
    Average_use_per_GB_DatacenterMJ = Average_use_per_GB_DatacenterkWh*3.6*3
    
    # MJ/MB
    Average_use_per_GBMB = Average_use_per_GBMJ/1024
    Average_use_per_GB_UserMB = Average_use_per_GB_UserMJ/1024
    Average_use_per_GB_UpstreamMB = Average_use_per_GB_UpstreamMJ/1024
    Average_use_per_GB_DatacenterMB = Average_use_per_GB_DatacenterMJ/1024
    
    # ------------------------------------- Estimated Energy Consumption ----------------------------------- #
    # Data usage per item: Assumes visiting 10 pages within a vendor (e.g., landing, search, item selection, etc.) 
    
    # Energy usage per item
    Energy_Usage_per_Item = Energy_Usage_Selection* 3.6 * 3/1024 * Data_Usage_Item # MJ/item
    
    # ------------------------------------------- Outputs --------------------------------------------------- #
    # MJ/item
    Average_use_per_GBItem = Average_use_per_GBMB * Data_Usage_Item
    Average_use_per_GB_UserItem = Average_use_per_GB_UserMB * Data_Usage_Item
    Average_use_per_GB_UpstreamItem = Average_use_per_GB_UpstreamMB * Data_Usage_Item
    Average_use_per_GB_DatacenterItem = Average_use_per_GB_DatacenterMB * Data_Usage_Item
    
    # User + Upstream For Etail
    User_Upstream = Energy_Usage_per_Item * ((USER[0]/GW_YEAR[0])+(UPSTREAM[0]/GW_YEAR[0])) # MJ/item
    
    # Upstream For Retail
    Upstream = Energy_Usage_per_Item * (UPSTREAM[0]/GW_YEAR[0])
    
    # Data Center For Retail and Etail
    Data_Center = Energy_Usage_per_Item * (DATA_CENTER[0]/GW_YEAR[0])
    
    return(User_Upstream,Upstream,Data_Center) 
    # ------------------------------------------------------------------------------------------------------- #
    

### 3.7. Last Mile

The assumptions in this section relate to the number of packages in a truck as well as the total distance of the delivery route.  In essence, the energy per delivered package is just the fraction of miles represented for an individual package.  The ranges of each of these two parameters are effectively educated guesses based on work by the research team. 

References:
Jaller, M. and A. Pahwa (2020). Evaluating the Environmental Impacts of Online Shopping: A Behavioral and Transportation Approach. Journal of Transportation Research Part D: Transport and Environment (in print)
Jaller, M., L. Pineda and H. Ambrose (2018) "Evaluating the Use of Zero-Emission Vehicles in Last Mile Deliveries." UC Davis Institute of Transport Studies, from https://escholarship.org/uc/item/7kr753nm.


In [ ]:
def LastMile(Year_selected,Scenario):
    
    # Call function
    Energy_mile_truck = Enpertmi(Year_selected,Scenario)
    # --------------------------------------- Energy per item ------------------------------------------------ #
    # Energy per item
    Energy_Item = (Distance_Delivery_Route['VMT'][Year_selected]) * Energy_mile_truck / (Basket_size_online)

    return(Energy_Item)
    # -------------------------------------------------------------------------------------------------------- #


### 3.8. Customer Transport

Parameters in this section all relate to trips driven in personal vehicles needed to purchase items from retail, or to return items purchased online (e.g., to a bricks and mortar store or post office).  Ranges of roundtrip miles driven and passengers per vehicle come from US federal transportation surveys and team research. Fuel economy ranges for personal vehicles are assumed but based on EPA data.  

“Basket sizes” are based on assumptions given a lack of data (assume only 1 item returned for online purchases) and are represented as fairly small numbers given the goal of comparing systems with an “online focus” – since online purchase baskets seem to be fairly small, whereas a “retail focus” would likely have a much larger basket size for shopping trips.

References:
Jaller, M. and A. Pahwa (2020). Evaluating the Environmental Impacts of Online Shopping: A Behavioral and Transportation Approach. Journal of Transportation Research Part D: Transport and Environment (in print)
Basket size source: https://vend-corp-cms.s3.amazonaws.com/Guides/Guide-Content/Vends-Retail-Benchmarks-Guide-2019.pdf


In [ ]:
def CustomerTransport(Year_selected,Scenario):
    
    # Call functions
    CarbonContents(Year_selected)

    # --------------------------------------- ENERGY PER MILE ----------------------------------------------- #
     
    # Energy Intensity (MJ/gal)
    Energy_Intensity = BTU_gal[0] * 1055 / 1000000
    
    # Carbon intensity (kg CO2/gal)
    #Carbon_Intensity_kgCO2gal = Energy_Intensity * Combustion_Emissions[0] / 1000
    
    #Carbon_Intensity_MJmi = Energy_Intensity / Fleet_Fuel_Economy
    Carbon_Intensity_MJmi = Report_Energy.loc[21,Year_selected]
    
    # ----------------------------------------- Outputs ----------------------------------------------------- #
    # ----------------------------------- ENERGY FOR SHOPPING TRIPS ----------------------------------------- #
    # Energy Intensity for Shopping trips per item MJ/item
    Energy_Intensity_Shopping_Trips_Item = Shopping_Trip_Distance['VMT'][Year_selected] * Carbon_Intensity_MJmi / Basket_Size_InStore
    Total_Energy_Intensity_Shoping_Trips = Total_VMT.loc[0,Year_selected] * Carbon_Intensity_MJmi / Basket_Size_InStore

    # Emissions for shopping trips kg CO2/item    
    # Emissions_Shopping_Trips = ((Energy_Intensity_Shopping_Trips_Item * Carbon_Intensity_kgCO2gal)/ Energy_Intensity)
    Emissions_Shopping_Trips = CO2_lastmile.loc[0,str(Year_selected)] / (Basket_Size_InStore * 1000) # gCO2 to kg, per item per person
    
    #Total_Emissions_Intensity_Shoping_Trips = (Total_Energy_Intensity_Shoping_Trips * Carbon_Intensity_kgCO2gal)/Energy_Intensity
    Total_Emissions_Intensity_Shoping_Trips = Total_CO2_file.loc[0,Year_selected] / (Basket_Size_InStore * 1000)

    # Energy instensity of average shopping trip MJ/trip
    Energy_Instensity_Average_Shopping_Trip = Scenario.med[19] * Carbon_Intensity_MJmi / (Scenario.med[19] * Scenario.med[19])

    # Air pollutants 
    NOX_Emissions_Shopping_Trips = NOX_lastmile.loc[0,str(Year_selected)] / (Basket_Size_InStore * 1000) # gCO2 to kg, per item per person
    PM25_Emissions_Shopping_Trips = PM25_lastmile.loc[0,str(Year_selected)] / (Basket_Size_InStore * 1000) # gCO2 to kg, per item per person
    SO2_Emissions_Shopping_Trips = SO2_lastmile.loc[0,str(Year_selected)] / (Basket_Size_InStore * 1000) # gCO2 to kg, per item per person

    return(Energy_Intensity_Shopping_Trips_Item, Emissions_Shopping_Trips, Energy_Instensity_Average_Shopping_Trip,
           Total_Energy_Intensity_Shoping_Trips,Total_Emissions_Intensity_Shoping_Trips, NOX_Emissions_Shopping_Trips, 
          PM25_Emissions_Shopping_Trips, SO2_Emissions_Shopping_Trips)
    # ------------------------------------------------------------------------------------------------------- #



### 3.9. Retail and Warehouse

Assumptions in this section use data on retail store and warehouse energy use to provide ranges of energy use per item shipped. The retail energy comes from US DOE Commercial Building Energy Consumption Survey (CBECS) 2012 dataset, and follows past work done by the authors on warehouse energy use and e-commerce shipments performed in 2008.


In [ ]:
def Retail_(Year_selected):
    # -------------------------------------Annual Values ---------------------------------------------------- #
    #  2012 CBECS Data; https://www.eia.gov/consumption/commercial/data/2012/c&e/cfm/c30.php. Low average and high values for the  regions in the US
    Nat_Gas_Use_cuft = np.array([17, (17+32)/2, 32]) # cu ft/sq ft for low, med, high
    Nat_Gas_Use_BTU = Nat_Gas_Use_cuft * 1030 # BTU/sq ft low, med, high
    Nat_Gas_Use_Joule = Nat_Gas_Use_BTU * 1055 # Joule/sq ft low, med, high
    Nat_Gas_Use_MJ = Nat_Gas_Use_Joule / 1000000 # MJ/sq ft low, med, high
    
    # Similar to above, CBECS Data; https://www.eia.gov/consumption/commercial/data/2012/c&e/cfm/c20.php
    Electricity_Use_kWh = np.array([13.5, (13.5+20.5)/2, 20.5]) # kWh/sq ft for low, med, high
    Electricity_Use_MJe = Electricity_Use_kWh * 3.6 # MJe/sf for low, med, high
    
    # Target Estimates
    Total_Annual_Target_Revenue = 74 # (billions)
    Avg_Price_Item = 23 # $
    Tot_Items_Sold = Total_Annual_Target_Revenue / Avg_Price_Item # (billions)
    Avg_Items_Bulk_Carton_Shipped = 25
    Total_Annual_Target_Cartons_Shipped = Tot_Items_Sold/Avg_Items_Bulk_Carton_Shipped  # (billions)
    E_commerce_Sales_astore = 0.05 # rate
    E_commerce_Sales_FC = 0.05 # rate
    
    # ---------------------------------- Estimated numbers for Retail---------------------------------------- #
    Retail_Stores_Number = 1844 # Total Store sq ft 
    Retail_Stores_Avg_Size = 130000 # sq ft
    NOxRates = [0.000181818,0.000227273,0.000363636] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    PMRates = [7.09091E-06,1.42727E-05,0.000420773] # https://www.epa.gov/sites/default/files/2020-07/documents/draft_egrid_pm_white_paper_7-20-20.pdf
    SO2Rates = [6.81818E-06,1.72727E-05,2.59091E-05] # https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.pdf
    
    # Average Annual Energy Use Estimates per facility (MJ)
    E_Nat_Gas_Use_Ave_Anual_FacilityMJ = Nat_Gas_Use_MJ * Retail_Stores_Avg_Size # MJ
    E_Electricity_Use_Anual_FacilitykWh = Electricity_Use_kWh * Retail_Stores_Avg_Size # kWh
    E_Electricity_Use_Anual_FacilityMJ = Electricity_Use_MJe * Retail_Stores_Avg_Size # MJ
    Energy_Use = E_Nat_Gas_Use_Ave_Anual_FacilityMJ + E_Electricity_Use_Anual_FacilityMJ
    
    # Average Annual Carbon Emission Estimates per facility (kg CO2e)
    C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.056
    C_Electricity_Use_Anual_FacilitykgCO2 = E_Electricity_Use_Anual_FacilitykWh * [60/2.2/1000, 940/2.2/1000, 2000/2.2/1000]
    C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.000006
    C_Electricity_Use_Anual_FacilitykgNOX = E_Electricity_Use_Anual_FacilitykWh * NOxRates
    C_Nat_Gas_Use_Ave_Anual_FacilitykgPM = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000001
    C_Electricity_Use_Anual_FacilitykgPM = E_Electricity_Use_Anual_FacilitykWh * PMRates
    C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 = E_Nat_Gas_Use_Ave_Anual_FacilityMJ * 0.0000003
    C_Electricity_Use_Anual_FacilitykgSO2 = E_Electricity_Use_Anual_FacilitykWh * SO2Rates
    Carbon_Use = C_Nat_Gas_Use_Ave_Anual_FacilitykgCO2 + C_Electricity_Use_Anual_FacilitykgCO2
    Carbon_UseNOx = C_Nat_Gas_Use_Ave_Anual_FacilitykgNOX + C_Electricity_Use_Anual_FacilitykgNOX
    Carbon_UsePM = C_Nat_Gas_Use_Ave_Anual_FacilitykgPM + C_Electricity_Use_Anual_FacilitykgPM
    Carbon_UseSO2 = C_Nat_Gas_Use_Ave_Anual_FacilitykgSO2 + C_Electricity_Use_Anual_FacilitykgSO2
    
    # ------------------------- Range of Annual Shipments "flow through of total" per facility -------------- #
    Summary_Range_Used_perce = np.array([1,(1+0.8)/2,0.8])*(Tot_Items_Sold*(1-E_commerce_Sales_FC))*10**9/Retail_Stores_Number
    
    # -------------------------------------- Outputs -------------------------------------------------------- #
    # Energy Use Per ITEM Shipped (MJ/item) - note: the denominators are in reverse order to preserve "best/worst" #
    Energy_Use_Item_Shipped = Energy_Use / Summary_Range_Used_perce # MJ/item

    # Carbon emissions Per ITEM Shipped (kg CO2/item) - for g CO2, multiply by 1000!
    Carbon_Emissions_Item_Shipped = Carbon_Use / Summary_Range_Used_perce # kg CO2/item
    global Carbon_emissions_Per_Item_ShippedNOx
    Carbon_emissions_Per_Item_ShippedNOx = Carbon_UseNOx / Summary_Range_Used_perce
    global Carbon_emissions_Per_Item_ShippedPM
    Carbon_emissions_Per_Item_ShippedPM = Carbon_UsePM / Summary_Range_Used_perce
    global Carbon_emissions_Per_Item_ShippedSO2
    Carbon_emissions_Per_Item_ShippedSO2 = Carbon_UseSO2 / Summary_Range_Used_perce

    return(Carbon_Emissions_Item_Shipped)
    # ------------------------------------------------------------------------------------------------------- #


### 3.10 Headquarters (HQ) Energy Use

The data on this section was from a study done by Weber at al. (from Carnegie Mellon University) in 2008 for the e-tailer buy.com  (later bought by rakuten)

This section collects assumptions and other parameters needed to estimate the effects associated with the higher-level (management related) activities of these companies - these would be accounting, oversight, etc.  


In [ ]:
def HQ():
    # --------------------- Headquarters Electricity Use - collected data from buy.com ---------------------- #
    Total_Building_Usage = 2405108
    Allocated_Usage = 539045.70 #Buy.com
    Months_with_electricity_data = 13
    Avg_Use_per_Month_Total_Building = Total_Building_Usage/Months_with_electricity_data
    Avg_Use_per_Month_Allocated = Allocated_Usage/Months_with_electricity_data
    Avg_Energy_Use_per_Month_Allocated = Avg_Use_per_Month_Allocated*3.6 # (MJ/m)
    Avg_Days_per_Month = 30
    Avg_Energy_Use_per_Day = Avg_Energy_Use_per_Month_Allocated/Avg_Days_per_Month #(MJ/d)
    Total_Buy_com_shimpments_sample = 280931
    Total_Building_Area = 111103 #(sqft)
    Buy_com_Area = 24901 #(sqft)
    Avg_Energy_Use_per_Shipment = Avg_Energy_Use_per_Month_Allocated/Total_Buy_com_shimpments_sample #(MJ/Shipment)
    
    # ------------------- Headquarters Natural Gas Use - collected data from buy.com ------------------------ #
    Gas_Use = 808024.50 # MJ
    Months_with_NG_data = 5
    Avg_Monthly_Natural_Gas_Use = Gas_Use/Months_with_NG_data
    Avg_Days_per_Month = 30.5
    Avg_NG_Use_per_Day = Avg_Monthly_Natural_Gas_Use/Avg_Days_per_Month # (MJ/d)
    Average_use_per_shipment = Avg_Monthly_Natural_Gas_Use/Total_Buy_com_shimpments_sample #(MJ/Shipment)
    
    return(Avg_Energy_Use_per_Shipment,Average_use_per_shipment)
    # ------------------------------------------------------------------------------------------------------- #

# 4. Summary

Summary provides results from a specific combination of retail scenario, energy scenario, MSA and growth population. 

## 4.1. Energy per item

In [ ]:
# ---------------------------------------- MJ/item ----------------------------------------------------------- #
Scenario1 = Purchasing_Scenarios(str(option))  
global Total_Energy
Total_Energy = []
def MJ(j,Year_selected,option,MSA_op,Scenario,Energy_option):
    
    # Call functions
    Market_Population = MSA_Market_Population(str(MSA_op)) 
    Rates(str(Energy_option)) 
    Scenario_Level(Scenario)
    Retail_percentage = Retail_Online_perc(Year_selected,Market_Population,str(option))
    Carbon_emissions_Etail = WarehouseEtail(Year_selected)
    Pack = Packaging(Year_selected)
    Distance = Distances(Year_selected,Scenario)
    DataUsag = DataUsage()
    Energy_per_item = LastMile(Year_selected,Scenario)
    Energy_For_Shopping = CustomerTransport(Year_selected, Scenario)
    HQ_Average_Use = HQ()
    
    # MJ/Item - Rush
    Flag_Multiplier_LH = 1
    Flag_Multiplier_LM = 1

    # Results of MJ per Item
    class Retail_MJ:
        Data_Centers_HQ = (HQ_Average_Use[0]+HQ_Average_Use[1])*(1+Rate_Instore_Purchase)
        Warehouse_Energy = Retail_Warehouse_energy_use * MJ_Warehouse[0][str(Year_selected)] 
        Freight_Energy_Road_only = R_Freight_Energy_Road[Select_Shipping_efficiency]
        Freight_Energy_Air_Only = 0.0
        Freight_Energy = Freight_Energy_Road_only+Freight_Energy_Air_Only
        Retail_Store_Energy = Retail_energy_use * (1+Rate_Instore_Purchase)
        Packaging = Pack[2][Energy_Emission_per_packaging_material_Retail]
        Last_Mile_Delivery = 0.0
        Customer_Transport = Energy_For_Shopping[0] * (1+Rate_Instore_Purchase)
        Computer_Network = 0.0
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total
        Total_Etail_Air = Data_Centers_HQ + Freight_Energy_Air_Only + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network

    class Ecommerce_MJ:
        Data_Centers_HQ = (HQ_Average_Use[0]+HQ_Average_Use[1]+DataUsag[2])*(1+Rate_Online_Purchase)
        Warehouse_Energy = Etail_Warehouse_energy_use*(1+Rate_Online_Purchase)*MJ_Warehouse[0][str(Year_selected)] 
        Freight_Energy_Road_only = E_Freight_Energy_Road[Select_Shipping_efficiency]
        Freight_Energy_Air_Only = E_Freight_Energy_Air[Select_Shipping_efficiency]*MJ_Aviation[0][str(Year_selected)]
        Freight_Energy = (Freight_Energy_Road_only+Freight_Energy_Air_Only)*(1+Flag_Multiplier_LH*(Distance[Select_Level_of_Rush_Shipments]-1))
        Retail_Store_Energy = 0.0
        Packaging = (Pack[3][Energy_Emission_per_packaging_material_Etail])*(1+Rate_Online_Purchase)
        Last_Mile_Delivery = Energy_per_item*(1+Rate_Online_Purchase)*(1+Flag_Multiplier_LH*(Distance[Select_Level_of_Rush_Shipments]-1))
        Customer_Transport = Energy_For_Shopping[0] * (Rate_Online_Purchase)
        Computer_Network = DataUsag[0]*(1+Rate_Online_Purchase)
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Data_Centers_HQ + Freight_Energy_Road_only + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport 
        Total_Etail_Air = Data_Centers_HQ + Freight_Energy_Air_Only + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
    
    class Omnichannel_MJ:
        Data_Centers_HQ = Retail_MJ.Data_Centers_HQ*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Data_Centers_HQ
        Warehouse_Energy = Retail_MJ.Warehouse_Energy*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Warehouse_Energy
        Freight_Energy_Road_only = Retail_MJ.Freight_Energy_Road_only*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Freight_Energy_Road_only
        Freight_Energy_Air_Only = Retail_MJ.Freight_Energy_Air_Only*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Freight_Energy_Air_Only
        Freight_Energy = Retail_MJ.Freight_Energy*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Freight_Energy
        Retail_Store_Energy = Retail_MJ.Retail_Store_Energy*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Retail_Store_Energy
        Packaging = Retail_MJ.Packaging*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Packaging
        Last_Mile_Delivery = Retail_MJ.Last_Mile_Delivery*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Last_Mile_Delivery
        Customer_Transport = Retail_MJ.Customer_Transport*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Customer_Transport
        Computer_Network = Retail_MJ.Computer_Network*Retail_percentage+(1-Retail_percentage)*Ecommerce_MJ.Computer_Network
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Data_Centers_HQ + Freight_Energy_Road_only + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport 
        Total_Etail_Air = Data_Centers_HQ + Freight_Energy_Air_Only + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
       
    # Saveing parameters for CO2 function
    global MJRatil_Ecommerce
    MJRatil_Ecommerce = [Retail_MJ.Freight_Energy_Road_only,Retail_MJ.Freight_Energy_Air_Only,
                         Ecommerce_MJ.Freight_Energy_Road_only,Ecommerce_MJ.Freight_Energy_Air_Only,
                         Ecommerce_MJ.Last_Mile_Delivery,Ecommerce_MJ.Computer_Network]

    # Saving results
    if option == "1":                    # Retail
        MJ_output = np.array([Retail_MJ.Data_Centers_HQ,Retail_MJ.Warehouse_Energy,Retail_MJ.Freight_Energy,
                        Retail_MJ.Retail_Store_Energy,Retail_MJ.Packaging,Retail_MJ.Last_Mile_Delivery,
                       Retail_MJ.Customer_Transport,Retail_MJ.Computer_Network,Retail_MJ.Total])
    elif option == "2" or option == "3": # Online
        MJ_output = np.array([Ecommerce_MJ.Data_Centers_HQ,Ecommerce_MJ.Warehouse_Energy,Ecommerce_MJ.Freight_Energy,
                        Ecommerce_MJ.Retail_Store_Energy,Ecommerce_MJ.Packaging,Ecommerce_MJ.Last_Mile_Delivery,
                       Ecommerce_MJ.Customer_Transport,Ecommerce_MJ.Computer_Network,Ecommerce_MJ.Total])
    else:                                # Both
        MJ_output = np.array([Omnichannel_MJ.Data_Centers_HQ,Omnichannel_MJ.Warehouse_Energy,Omnichannel_MJ.Freight_Energy,
                        Omnichannel_MJ.Retail_Store_Energy,Omnichannel_MJ.Packaging,Omnichannel_MJ.Last_Mile_Delivery,
                       Omnichannel_MJ.Customer_Transport,Omnichannel_MJ.Computer_Network,Omnichannel_MJ.Total])
    
    
    return(MJ_output)

# ----------------------------------------------Print the results----------------------------------------------#
Total_Energy = []
MJ_result = []
MJItem = []

j=0
for i in Years:
    # Energy per item
    MJ_call = MJ(j,i,option,MSA_op,Scenario1,Energy_option)
    MJ_result.append(MJ_call)
    MJItem.append(MJRatil_Ecommerce)
    del MJRatil_Ecommerce
    
    # Total Energy (TMJ) per day
    Energy_For_Shopping = CustomerTransport(i,Scenario1)
    TMJ = MJ_call * Number_Items[j]
    #TMJ[6] = Energy_For_Shopping[3] * (1+Rate_Instore_Purchase)
    Total_Energy.append(TMJ) # Saving every year results
    j=j+1

# --------------------------------------------------- Table ----------------------------------------------------#
Columns = ['Data_Centers_HQ','Warehouse_Energy','Freight_Energy','Retail_Store_Energy','Packaging',
                                      'Last_Mile_Delivery','Customer_Transport','Computer_Network','Total']
# Energy per item
T = np.transpose(MJ_result)
MJ_table = pd.DataFrame(T[8,])
MJ_table.columns=[Title]
MJ_table['Year'] = Years
MJ_table.set_index('Year', inplace=True)

# Total energy
Total_Energy_day1 = pd.DataFrame(Total_Energy, columns=Columns)
Total_Energy_day1['Years']=Years
Total_Energy_day1.set_index('Years', inplace=True)
del Total_Energy

# Print both
MJ_table['Energy Day'] = Total_Energy_day1['Total']

# ----------------------------------------- Table All Parameters ---------------------------------------------#
T1MJ = pd.DataFrame(MJ_result, columns=Columns)
T1MJ['Years']=Years
T1MJ.set_index('Years', inplace=True)
T2MJ = T1MJ.drop(columns=['Total'])

# ----------------------------------------------- Total Energy per Day ---------------------------------------#
Total_Energy_day = Total_Energy_day1.drop(columns=['Total'])
# ----------------------------------------------------------------------------------------------------------- #

## 4.2. Emissions per item

In [ ]:
# ---------------------------------------- CO2/item --------------------------------------------------------- #
def CO2(i,Year_selected,option,MSA_op,Scenario,Energy_option):
    # CO2/Item
    Flag_Multiplier_LH = 1
    Flag_Multiplier_LM = 1

    # Call functions
    Market_Population = MSA_Market_Population(str(MSA_op)) 
    Rates(str(Energy_option)) 
    #VMT_2(str(MSA_op),Year_selected) 
    Scenario_Level(Scenario)
    Retail_per = Retail_Online_perc(Year_selected,Market_Population,str(option))
    Carbon_emissions_Etail = WarehouseEtail(Year_selected)
    Carbon_emissions_Omni=np.array(WarehouseRetail(Year_selected))
    Pack = Packaging(Year_selected)
    DataUsag = DataUsage()
    Retail_Carbon_Emissions = Retail_(Year_selected)
    Energy_For_Shopping = CustomerTransport(Year_selected,Scenario)
    HQ_Average_Use = HQ()
    Distance = Distances(Year_selected,Scenario)
    
    if option == 3:
        Carbon_emissions_Omni[0][O_count]=Carbon_emissions_Etail[A_count].copy() 

    # Results of CO2 per Item
    class Retail_CO2:
        Data_Centers_HQ = (HQ_Average_Use[0]*grCO2e_MJe[0]+HQ_Average_Use[1]*Combustion_Emissions[6])*(1+Rate_Instore_Purchase)
        Warehouse_Energy = Carbon_emissions_Omni[1][O_count] * 1000 * CO2_Warehouse[0][str(Year_selected)] 
        Freight_Energy_Road_only = MJItem[i][0] * Combustion_Emissions[3]*Electricity_share_of_ETrucks[0][str(Year_selected)]
        Freight_Energy_Air_Only = MJItem[i][1] * Combustion_Emissions[2]*CO2_Aviation[0][str(Year_selected)]
        Freight_Energy = Freight_Energy_Road_only+Freight_Energy_Air_Only
        Retail_Store_Energy = Retail_Carbon_Emissions[R_count] * (1+Rate_Instore_Purchase) * 1000 * CO2_Warehouse[0][str(Year_selected)]
        Packaging = Pack[0][Energy_Emission_per_packaging_material_Retail]
        Last_Mile_Delivery = 0.0
        Customer_Transport = Energy_For_Shopping[1] * (1+Rate_Instore_Purchase) * 1000 * Electricity_share_of_EV[0][str(Year_selected)]
        Computer_Network = 0.0
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total
        Total_Etail_Air = Total + Freight_Energy_Air_Only - Freight_Energy
    
    class Ecommerce_CO2:
        Data_Centers_HQ = (HQ_Average_Use[0]*grCO2e_MJe[0]+HQ_Average_Use[1]*Combustion_Emissions[6]+DataUsag[2]*grCO2e_MJe[0])*(1+Rate_Online_Purchase)
        Warehouse_Energy = Carbon_emissions_Omni[0][O_count] * 1000 * (1+Rate_Online_Purchase) * CO2_Warehouse[0][str(Year_selected)]
        Freight_Energy_Road_only = MJItem[i][2] * Combustion_Emissions[3]*Electricity_share_of_ETrucks[0][str(Year_selected)]
        Freight_Energy_Air_Only = MJItem[i][3] * Combustion_Emissions[2]*CO2_Aviation[0][str(Year_selected)]
        Freight_Energy = (Freight_Energy_Road_only+Freight_Energy_Air_Only)*(1+Flag_Multiplier_LH*(Distance[Select_Level_of_Rush_Shipments]-1))
        Retail_Store_Energy = 0.0
        Packaging = (Pack[1][Energy_Emission_per_packaging_material_Etail])*(1+Rate_Online_Purchase)
        #Last_Mile_Delivery = MJItem[i][4] * Combustion_Emissions[3]*Electricity_share_of_Commercial[0][str(Year_selected)]
        Last_Mile_Delivery = (CO2_lastmile.loc[1,str(Year_selected)])*Electricity_share_of_Commercial[0][str(Year_selected)]
        Customer_Transport = Energy_For_Shopping[1] * (Rate_Online_Purchase) * 1000 * Electricity_share_of_EV[0][str(Year_selected)]
        Computer_Network = MJItem[i][5] * Combustion[0]* 0.4535/3.6
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total + Freight_Energy_Road_only - Freight_Energy
        Total_Etail_Air = Total + Freight_Energy_Air_Only - Freight_Energy

    class Omnichannel_CO2:
        Data_Centers_HQ = Retail_CO2.Data_Centers_HQ*Retail_per+(1-Retail_per)*Ecommerce_CO2.Data_Centers_HQ
        Warehouse_Energy = Retail_CO2.Warehouse_Energy*Retail_per+(1-Retail_per)*Ecommerce_CO2.Warehouse_Energy
        Freight_Energy_Road_only = Retail_CO2.Freight_Energy_Road_only*Retail_per+(1-Retail_per)*Ecommerce_CO2.Freight_Energy_Road_only
        Freight_Energy_Air_Only = Retail_CO2.Freight_Energy_Air_Only*Retail_per+(1-Retail_per)*Ecommerce_CO2.Freight_Energy_Air_Only
        Freight_Energy = Retail_CO2.Freight_Energy*Retail_per+(1-Retail_per)*Ecommerce_CO2.Freight_Energy
        Retail_Store_Energy = Retail_CO2.Retail_Store_Energy*Retail_per+(1-Retail_per)*Ecommerce_CO2.Retail_Store_Energy
        Packaging = Retail_CO2.Packaging*Retail_per+(1-Retail_per)*Ecommerce_CO2.Packaging
        Last_Mile_Delivery = Retail_CO2.Last_Mile_Delivery*Retail_per+(1-Retail_per)*Ecommerce_CO2.Last_Mile_Delivery
        Customer_Transport = Retail_CO2.Customer_Transport*Retail_per+(1-Retail_per)*Ecommerce_CO2.Customer_Transport 
        Computer_Network = Retail_CO2.Computer_Network*Retail_per+(1-Retail_per)*Ecommerce_CO2.Computer_Network
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total + Freight_Energy_Road_only - Freight_Energy
        Total_Etail_Air = Total + Freight_Energy_Air_Only - Freight_Energy
    
    # Saving results
    if option == "1":                    # Retail
        CO2_output = np.array([Retail_CO2.Data_Centers_HQ,Retail_CO2.Warehouse_Energy,Retail_CO2.Freight_Energy,
                            Retail_CO2.Retail_Store_Energy,Retail_CO2.Packaging,Retail_CO2.Last_Mile_Delivery,
                           Retail_CO2.Customer_Transport,Retail_CO2.Computer_Network,Retail_CO2.Total])
    elif option == "2" or option == "3": # Online
        CO2_output = np.array([Ecommerce_CO2.Data_Centers_HQ,Ecommerce_CO2.Warehouse_Energy,Ecommerce_CO2.Freight_Energy,
                        Ecommerce_CO2.Retail_Store_Energy,Ecommerce_CO2.Packaging,Ecommerce_CO2.Last_Mile_Delivery,
                       Ecommerce_CO2.Customer_Transport,Ecommerce_CO2.Computer_Network,Ecommerce_CO2.Total])
    else:                                # Both
        CO2_output = np.array([Omnichannel_CO2.Data_Centers_HQ,Omnichannel_CO2.Warehouse_Energy,Omnichannel_CO2.Freight_Energy,
                        Omnichannel_CO2.Retail_Store_Energy,Omnichannel_CO2.Packaging,Omnichannel_CO2.Last_Mile_Delivery,
                       Omnichannel_CO2.Customer_Transport,Omnichannel_CO2.Computer_Network,Omnichannel_CO2.Total])
    
    
    return(CO2_output)
# ------------------------------------------------Print the results-------------------------------------------#
# Table 
Total_CO2 = []
CO2_result = []

j = 0
for i in Years:
    # Co2 per item
    CO2_call = CO2(j,i,option,MSA_op,Scenario1,Energy_option)
    CO2_result.append(CO2_call)
    
    # Total CO2 Emissions (TCO2) per day
    Energy_For_Shopping = CustomerTransport(i,Scenario1)
    TCO2 = CO2_call * Number_Items[j]
    TCO2[6] = Energy_For_Shopping[4] * (1+Rate_Instore_Purchase) * 1000
    Total_CO2.append(TCO2)
    j=j+1
# ----------------------------------------------Emissions Table ----------------------------------------------#
Columns=['Data_Centers_HQ','Warehouse_Energy','Freight_Energy','Retail_Store_Energy','Packaging',
                                      'Last_Mile_Delivery','Customer_Transport','Computer_Network','Total']
C = np.transpose(CO2_result)
CO2_table = pd.DataFrame(C[8,])
CO2_table.columns=[Title]
CO2_table['Year'] = Years
CO2_table.set_index('Year', inplace=True)

# Total energy
Total_CO21 = pd.DataFrame(Total_CO2, columns=Columns)
Total_CO21['Years']=Years
Total_CO21.set_index('Years', inplace=True)

# ---------------------------------------------- Stacked bar table -------------------------------------------#
T1CO2 = pd.DataFrame(CO2_result)
T1CO2.columns = Columns
T1CO2['Years']=Years
T1CO2.set_index('Years', inplace=True)
T2CO2 = T1CO2.drop(columns=['Total'])

# ----------------------------------------------- Total Energy per Day ---------------------------------------#
Total_CO2 = Total_CO21.drop(columns=['Total'])
# ------------------------------------------------------------------------------------------------------------#


## 4.3. Criteria Air Poluttant per item

In [ ]:
# ---------------------------------------- CO2/item --------------------------------------------------------- #
def Air(i,Year_selected,option,MSA_op,Scenario,Energy_option,Air_lastmile,Customer,gr_MJeData,Combustion_Pol,Total_Carbon_,gr_MJeWareH,
        PackagingRetail,PackagingEtail,Carbon_emissions_Per_Item_Shipped):
    # Air/Item
    Flag_Multiplier_LH = 1
    Flag_Multiplier_LM = 1

    # Call functions
    Market_Population = MSA_Market_Population(str(MSA_op)) 
    Rates(str(Energy_option)) 
    #VMT_2(str(MSA_op),Year_selected) 
    Scenario_Level(Scenario)
    Retail_per = Retail_Online_perc(Year_selected,Market_Population,str(option))
    Carbon_emissions_Etail = WarehouseEtail(Year_selected)
    Carbon_emissions_Omni=np.array(WarehouseRetail(Year_selected))
    Pack = Packaging(Year_selected)
    DataUsag = DataUsage()
    Retail_Carbon_Emissions = Retail_(Year_selected)
    Energy_For_Shopping = CustomerTransport(Year_selected,Scenario)
    HQ_Average_Use = HQ()
    Distance = Distances(Year_selected,Scenario)
    
    if option == 3:
        Carbon_emissions_Omni[gr_MJeWareH-1][O_count]=Carbon_emissions_Etail[A_count].copy() 
    
    # Results of Air per Item
    class Retail_Air:
        Data_Centers_HQ = (HQ_Average_Use[0]*gr_MJeData+HQ_Average_Use[1]*Combustion_Pol[3])*(1+Rate_Instore_Purchase)
        Warehouse_Energy = Carbon_emissions_Omni[gr_MJeWareH][O_count] * 1000 * CO2_Warehouse[0][str(Year_selected)] 
        Freight_Energy_Road_only = MJItem[i][0] * Combustion_Pol[2] *Electricity_share_of_ETrucks[0][str(Year_selected)]
        Freight_Energy_Air_Only = MJItem[i][1] * Combustion_Pol[1]*CO2_Aviation[0][str(Year_selected)]
        Freight_Energy = Freight_Energy_Road_only+Freight_Energy_Air_Only
        Retail_Store_Energy = Carbon_emissions_Per_Item_Shipped[R_count] * (1+Rate_Instore_Purchase) * 1000 * CO2_Warehouse[0][str(Year_selected)]
        Packaging = PackagingRetail[Energy_Emission_per_packaging_material_Retail]
        Last_Mile_Delivery = 0.0
        Customer_Transport = Energy_For_Shopping[5] * (1+Rate_Instore_Purchase) * 1000 * Electricity_share_of_EV[0][str(Year_selected)]
        Computer_Network = 0.0
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total
        Total_Etail_Air = Total + Freight_Energy_Air_Only - Freight_Energy

    class Ecommerce_Air:
        Data_Centers_HQ = (HQ_Average_Use[0]*gr_MJeData+HQ_Average_Use[1]*Combustion_Pol[3]+DataUsag[2]*gr_MJeData)*(1+Rate_Online_Purchase)
        Warehouse_Energy = Total_Carbon_[O_count] * 1000 * (1+Rate_Online_Purchase) * CO2_Warehouse[0][str(Year_selected)]
        Freight_Energy_Road_only = MJItem[i][2] * Combustion_Pol[2]*Electricity_share_of_ETrucks[0][str(Year_selected)]
        Freight_Energy_Air_Only = MJItem[i][3] * Combustion_Pol[1]*CO2_Aviation[0][str(Year_selected)]
        Freight_Energy = (Freight_Energy_Road_only+Freight_Energy_Air_Only)*(1+Flag_Multiplier_LH*(Distance[Select_Level_of_Rush_Shipments]-1))
        Retail_Store_Energy = 0.0
        Packaging = (PackagingEtail[Energy_Emission_per_packaging_material_Etail])*(1+Rate_Online_Purchase)
        #Last_Mile_Delivery = MJItem[i][4] * Combustion_Emissions[3]*Electricity_share_of_Commercial[0][str(Year_selected)]
        Last_Mile_Delivery = (Air_lastmile)*Electricity_share_of_Commercial[0][str(Year_selected)]
        Customer_Transport = Energy_For_Shopping[Customer] * (Rate_Online_Purchase) * 1000 * Electricity_share_of_EV[0][str(Year_selected)]
        Computer_Network = MJItem[i][5] * NOX_Combustion* 0.4535/3.6
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total + Freight_Energy_Road_only - Freight_Energy
        Total_Etail_Air = Total + Freight_Energy_Air_Only - Freight_Energy

    class Omnichannel_Air:
        Data_Centers_HQ = Retail_Air.Data_Centers_HQ*Retail_per+(1-Retail_per)*Ecommerce_Air.Data_Centers_HQ
        Warehouse_Energy = Retail_Air.Warehouse_Energy*Retail_per+(1-Retail_per)*Ecommerce_Air.Warehouse_Energy
        Freight_Energy_Road_only = Retail_Air.Freight_Energy_Road_only*Retail_per+(1-Retail_per)*Ecommerce_Air.Freight_Energy_Road_only
        Freight_Energy_Air_Only = Retail_Air.Freight_Energy_Air_Only*Retail_per+(1-Retail_per)*Ecommerce_Air.Freight_Energy_Air_Only
        Freight_Energy = Retail_Air.Freight_Energy*Retail_per+(1-Retail_per)*Ecommerce_Air.Freight_Energy
        Retail_Store_Energy = Retail_Air.Retail_Store_Energy*Retail_per+(1-Retail_per)*Ecommerce_Air.Retail_Store_Energy
        Packaging = Retail_Air.Packaging*Retail_per+(1-Retail_per)*Ecommerce_Air.Packaging
        Last_Mile_Delivery = Retail_Air.Last_Mile_Delivery*Retail_per+(1-Retail_per)*Ecommerce_Air.Last_Mile_Delivery
        Customer_Transport = Retail_Air.Customer_Transport*Retail_per+(1-Retail_per)*Ecommerce_Air.Customer_Transport 
        Computer_Network = Retail_Air.Computer_Network*Retail_per+(1-Retail_per)*Ecommerce_Air.Computer_Network
        Total = Data_Centers_HQ + Warehouse_Energy + Freight_Energy + Retail_Store_Energy + Packaging + Last_Mile_Delivery + Customer_Transport + Computer_Network
        Total_Etail_Road = Total + Freight_Energy_Road_only - Freight_Energy
        Total_Etail_Air = Total + Freight_Energy_Air_Only - Freight_Energy
    
    # Saving results
    if option == "1":                    # Retail
        Air_output = np.array([Retail_Air.Data_Centers_HQ,Retail_Air.Warehouse_Energy,Retail_Air.Freight_Energy,
                            Retail_Air.Retail_Store_Energy,Retail_Air.Packaging,Retail_Air.Last_Mile_Delivery,
                           Retail_Air.Customer_Transport,Retail_Air.Computer_Network,Retail_Air.Total])
    elif option == "2" or option == "3": # Online
        Air_output = np.array([Ecommerce_Air.Data_Centers_HQ,Ecommerce_Air.Warehouse_Energy,Ecommerce_Air.Freight_Energy,
                        Ecommerce_Air.Retail_Store_Energy,Ecommerce_Air.Packaging,Ecommerce_Air.Last_Mile_Delivery,
                       Ecommerce_Air.Customer_Transport,Ecommerce_Air.Computer_Network,Ecommerce_Air.Total])
    else:                                # Both
        Air_output = np.array([Omnichannel_Air.Data_Centers_HQ,Omnichannel_Air.Warehouse_Energy,Omnichannel_Air.Freight_Energy,
                        Omnichannel_Air.Retail_Store_Energy,Omnichannel_Air.Packaging,Omnichannel_Air.Last_Mile_Delivery,
                       Omnichannel_Air.Customer_Transport,Omnichannel_Air.Computer_Network,Omnichannel_Air.Total])
    
    
    return(Air_output)


In [ ]:
# ------------------------------------------------NOX results-------------------------------------------#
# Table 
Total_NOX = []
NOX_result = []

j = 0
for i in Years:
    # NOX per item
    WarehouseRetail(Years)
    Retail_(Years)
    NOX_call = Air(j,i,option,MSA_op,Scenario1,Energy_option,NOX_lastmile.loc[1,str(i)],5,grNOX_MJe,Combustion_Pollutants[0],Total_CarbonNOx,3,
                   Packaging_item_gNOx_Retail,Packaging_item_gNOx_Etail,Carbon_emissions_Per_Item_ShippedNOx) # Customer transport = 5 means the position of NOX, Wharehouse = 2, the possition of NOX
    NOX_result.append(NOX_call)
    
    # Total NOX Emissions (TNOX) per day
    Energy_For_Shopping = CustomerTransport(i,Scenario1)
    TNOX = NOX_call * Number_Items[j]
    TNOX[6] = Energy_For_Shopping[4] * (1+Rate_Instore_Purchase) * 1000
    Total_NOX.append(TNOX)
    j=j+1
# ----------------------------------------------Emissions Table ----------------------------------------------#
Columns=['Data_Centers_HQ','Warehouse_Energy','Freight_Energy','Retail_Store_Energy','Packaging',
                                      'Last_Mile_Delivery','Customer_Transport','Computer_Network','Total']
C = np.transpose(NOX_result)
NOX_table = pd.DataFrame(C[8,])
NOX_table.columns=[Title]
NOX_table['Year'] = Years
NOX_table.set_index('Year', inplace=True)

# Total energy
Total_NOX1 = pd.DataFrame(Total_NOX, columns=Columns)
Total_NOX1['Years']=Years
Total_NOX1.set_index('Years', inplace=True)

# ---------------------------------------------- Stacked bar table -------------------------------------------#
T1NOX = pd.DataFrame(NOX_result)
T1NOX.columns = Columns
T1NOX['Years']=Years
T1NOX.set_index('Years', inplace=True)
T2NOX = T1NOX.drop(columns=['Total'])

# ----------------------------------------------- Total Energy per Day ---------------------------------------#
Total_NOX = Total_NOX1.drop(columns=['Total'])
# ------------------------------------------------------------------------------------------------------------#
pd.options.display.float_format = '{:.5f}'.format
display(T1NOX)

In [ ]:
# ------------------------------------------------PM2.5 results-------------------------------------------#
# Table 
Total_PM = []
PM_result = []

j = 0
for i in Years:
    # NOX per item
    WarehouseRetail(Years)
    PM_call = Air(j,i,option,MSA_op,Scenario1,Energy_option,PM25_lastmile.loc[1,str(i)],6,grPM_MJe,Combustion_Pollutants[0],Total_CarbonPM,5,
                   Packaging_item_gPM_Retail,Packaging_item_gPM_Etail,Carbon_emissions_Per_Item_ShippedPM) # Customer transport = 5 means the position of NOX, Wharehouse = 2, the possition of NOX
    PM_result.append(PM_call)
    
    # Total NOX Emissions (TNOX) per day
    Energy_For_Shopping = CustomerTransport(i,Scenario1)
    TPM = PM_call * Number_Items[j]
    TPM[6] = Energy_For_Shopping[4] * (1+Rate_Instore_Purchase) * 1000
    Total_PM.append(TPM)
    j=j+1
# ----------------------------------------------Emissions Table ----------------------------------------------#
Columns=['Data_Centers_HQ','Warehouse_Energy','Freight_Energy','Retail_Store_Energy','Packaging',
                                      'Last_Mile_Delivery','Customer_Transport','Computer_Network','Total']
C = np.transpose(PM_result)
PM_table = pd.DataFrame(C[8,])
PM_table.columns=[Title]
PM_table['Year'] = Years
PM_table.set_index('Year', inplace=True)

# Total energy
Total_PM1 = pd.DataFrame(Total_PM, columns=Columns)
Total_PM1['Years']=Years
Total_PM1.set_index('Years', inplace=True)

# ---------------------------------------------- Stacked bar table -------------------------------------------#
T1PM = pd.DataFrame(PM_result)
T1PM.columns = Columns
T1PM['Years']=Years
T1PM.set_index('Years', inplace=True)
T2PM = T1PM.drop(columns=['Total'])

# ----------------------------------------------- Total Energy per Day ---------------------------------------#
Total_PM = Total_PM1.drop(columns=['Total'])
# ------------------------------------------------------------------------------------------------------------#
pd.options.display.float_format = '{:.5f}'.format
display(T1PM)

In [ ]:
# ------------------------------------------------SO2 results-------------------------------------------#
# Table 
Total_SO2 = []
SO2_result = []

j = 0
for i in Years:
    # SO2 per item
    WarehouseRetail(Years)
    SO2_call = Air(j,i,option,MSA_op,Scenario1,Energy_option,SO2_lastmile.loc[1,str(i)],7,grSO2_MJe,Combustion_Pollutants[0],Total_CarbonSO2,7,
                   Packaging_item_gSO2_Retail,Packaging_item_gSO2_Etail,Carbon_emissions_Per_Item_ShippedSO2) # Customer transport = 5 means the position of NOX, Wharehouse = 2, the possition of NOX
    SO2_result.append(PM_call)
    
    # Total SO2 Emissions (TSO2) per day
    Energy_For_Shopping = CustomerTransport(i,Scenario1)
    TSO2 = SO2_call * Number_Items[j]
    TSO2[6] = Energy_For_Shopping[4] * (1+Rate_Instore_Purchase) * 1000
    Total_SO2.append(TSO2)
    j=j+1
# ----------------------------------------------Emissions Table ----------------------------------------------#
Columns=['Data_Centers_HQ','Warehouse_Energy','Freight_Energy','Retail_Store_Energy','Packaging',
                                      'Last_Mile_Delivery','Customer_Transport','Computer_Network','Total']
C = np.transpose(SO2_result)
SO2_table = pd.DataFrame(C[8,])
SO2_table.columns=[Title]
SO2_table['Year'] = Years
SO2_table.set_index('Year', inplace=True)

# Total energy
Total_SO21 = pd.DataFrame(Total_SO2, columns=Columns)
Total_SO21['Years']=Years
Total_SO21.set_index('Years', inplace=True)

# ---------------------------------------------- Stacked bar table -------------------------------------------#
T1SO2 = pd.DataFrame(SO2_result)
T1SO2.columns = Columns
T1SO2['Years']=Years
T1SO2.set_index('Years', inplace=True)
T2SO2 = T1PM.drop(columns=['Total'])

# ----------------------------------------------- Total Energy per Day ---------------------------------------#
Total_SO2 = Total_SO21.drop(columns=['Total'])
# ------------------------------------------------------------------------------------------------------------#
pd.options.display.float_format = '{:.5f}'.format
display(T1SO2)

## 4.3. Plot and Save to xlsx file

In [ ]:
# Save to a file
import io
import xlsxwriter
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook

# If you wish to write to more than one sheet in the workbook, it is necessary to specify an ExcelWriter object:
with pd.ExcelWriter(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option), engine='xlsxwriter') as writer:
    # Saving energy per item
    T1MJ.to_excel(writer, index=True, sheet_name='Energy_item')
    # Saving Total Energy
    Total_Energy_day1.to_excel(writer, index=True, sheet_name='Total_energy')
    # Saving emissions per item
    T1CO2.to_excel(writer, index=True, sheet_name='CO2_item')
    # Saving total emissions
    Total_CO21.to_excel(writer, index=True, sheet_name='Total_CO2')
    writer.close()

    
# ---------------------------------------------MJ Table ---------------------------------------------------------#    
termwidth, fillchar = 78, '='
print(' Total Energy per itme (MJ/item) '.center(termwidth, fillchar))
pd.options.display.float_format = '{:,.4f}'.format # format of tables display
display(MJ_table)

#------------------------------------------ Plot the Energy along the time --------------------------------------#
#plot each series with custom colors and labels
MJFig,ax_CO2 = plt.subplots()
ax_MJ = plt.plot(MJ_table[Title], color='blue')
#add legend
plt.legend(title=Title)
#add axes labels and a title
plt.ylabel('Energy per itme (MJ/item)', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.title('Results: Energy', fontsize=16)
#display plot
plt.show()

# Save plot 
imgdata = io.BytesIO()
MJFig.savefig(imgdata, format='png') # save in memory
# Load workbook
wb = openpyxl.load_workbook(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))
ws = wb['Energy_item']
# Add image to worksheet
img = openpyxl.drawing.image.Image(imgdata)
ws.add_image(img, 'A9')
# Save workbook
wb.save(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))

# ----------------------------------------- Table All Parameters ----------------------------------------------#
termwidth, fillchar = 78, '='
print(' Energy per item (MJ/item) for every component '.center(termwidth, fillchar))
display(T2MJ)

# ---------------------------------------------- Stacked bar plot ---------------------------------------------#

MJSacked = T2MJ.plot(kind='bar', stacked=True, colormap='tab10', figsize=(10, 6))
fig = MJSacked.get_figure()
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlabel("Year")
plt.ylabel("Energy per item (MJ/item)")
plt.tight_layout()
# Percentage inside the figure
#T_prop = []
#for i in Columns:
#    prop=T1[i]/T[8]
#    T_prop.append(prop)

#del T_prop[-1]
#T2_p=pd.DataFrame(T_prop)
#T2_prop = T2_p.rename(columns=Years)

#for n, x in enumerate([*T2.index.values]):
#    for proportion in T2_prop[x]:        
#        plt.text(x=n,
#                 y=proportion,
#                 s=f'{np.round(proportion * 100, 1)}%', 
#                 color="black",
#                 fontsize=12,
#                 fontweight="bold")
plt.show()

# Save plot 
imgdata = io.BytesIO()
fig.savefig(imgdata, format='png') # save in memory
# Load workbook
wb = openpyxl.load_workbook(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))
ws = wb['Energy_item']
# Add image to worksheet
img = openpyxl.drawing.image.Image(imgdata)
ws.add_image(img, 'L9')
# Save workbook
wb.save(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))

# ----------------------------------------------- Total Energy per Day ------------------------------------------#
termwidth, fillchar = 78, '='
print(' Energy per day (MJ/item) for every component '.center(termwidth, fillchar))
display(Total_Energy_day)

#---------------------------------------- Plot the Emissions along the time -------------------------------------#
# Print table
CO2_table['Emissions/Day'] = Total_CO21['Total']
pd.options.display.float_format = '{:,.2f}'.format # format of tables display
termwidth, fillchar = 78, '='
print(' Total Emissions per item (gr CO2/item) '.center(termwidth, fillchar))
display(CO2_table)  

# Plot each series with custom colors and labels
fig,ax_CO2=plt.subplots()
ax_CO2.plot(CO2_table[Title], color='red')
#add legend
ax_CO2.legend(title=Title)
#add axes labels and a title
plt.ylabel('Emissions per item (gr CO2/item)', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.title('Results: Emissions', fontsize=16)
#display plot
plt.show()

# Save plot 
imgdata = io.BytesIO()
fig.savefig(imgdata, format='png') # save in memory
# Load workbook
wb = openpyxl.load_workbook(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))
ws = wb['CO2_item']
# Add image to worksheet
img = openpyxl.drawing.image.Image(imgdata)
ws.add_image(img, 'A9')
# Save workbook
wb.save(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))

# -------------------------------------------Emissions Stacked bar table ---------------------------------------#
termwidth, fillchar = 78, '='
print(' Emissions per item (gr CO2/item) for each component '.center(termwidth, fillchar))
# Table
display(T2CO2)
# Figure
plot = T2CO2.plot(kind='bar', stacked=True, colormap='tab10', figsize=(10, 6))
fig = plot.get_figure()
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlabel("Year")
plt.ylabel("Emissions per item (gr CO2/item)")
plt.tight_layout()
plt.show()

# Save plot 
imgdata = io.BytesIO()
fig.savefig(imgdata, format='png') # save in memory
# Load workbook
wb = openpyxl.load_workbook(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))
ws = wb['CO2_item']
# Add image to worksheet
img = openpyxl.drawing.image.Image(imgdata)
ws.add_image(img, 'L9')
# Save workbook
wb.save(Path+'/Results/City_{}_{}_{}_Energy_{}.xlsx'.format(MSA_op, PG, Title, Energy_option))

# ----------------------------------------- Total Emissions per Day --------------------------------------------#
termwidth, fillchar = 78, '='
print('Emissions per day (CO2/item) for every component '.center(termwidth, fillchar))
display(Total_CO2)
# ------------------------------------------------------------------------------------------------------------- #
    

# 5. Analysis of multiple scenarios

## 5.1. Sensitivity Analysis – Tornado Diagram

In [ ]:
# --------------------------------------- Data from previus results ---------------------------------------------#
from matplotlib.lines import Line2D
Scenario2 = Scenario1.drop(columns=['Select_Level'])
Scenario2['Select_Level'] = np.zeros(len(Scenario2['Description']))
Number_parameters = len(Scenario2.Select_Level) 

# Base scenario
for s in range(Number_parameters):
    if s == 12:
        Scenario2.loc[s,'Select_Level'] = 'Heavy Duty Truck (25 ton cargo)'
    elif s == 14:
        Scenario2.loc[s,'Select_Level'] = 'Medium Truck (20,000 lb)'
    else:
        Scenario2.loc[s,'Select_Level'] = 'med'

MJ_base = []
CO2_base = []
for s in range(Number_parameters):
    MJItem = []
    MJ_result = []
    CO2_result = []
    n=0
    for k in Years:#-------------------------------------------- Energy
        MJ_call = MJ(n,k,option,MSA_op,Scenario2,Energy_option)
        MJ_result.append(MJ_call[8])
        MJItem.append(MJRatil_Ecommerce)
        n=n+1
        del MJRatil_Ecommerce
        del MJ_call
    l = 0
    for m in Years:# ------------------------------------------- Emissions
        CO2_call = CO2(l,m,option,MSA_op,Scenario2,Energy_option)
        CO2_result.append(CO2_call[8])
        l=l+1
        del CO2_call
        
    # Saving data
    MJ_base.append(MJ_result)
    CO2_base.append(CO2_result)
    # Deleting all values
    del MJ_result
    del CO2_result
    del MJItem
    # Restoring values


# Changing level of parameters (low-high)
Level = ['low','high']
Level_inv = ['high','low']
Freight_Truck = ['Heavy Duty Truck (25 ton cargo)','Medium Duty Truck (8 ton cargo)']
Last_Mile_Truck = ['Medium Truck (20,000 lb)', 'Light Truck (less than 10,000 lb Truck)','Heavy Trucks (more than 26,000 lb trucks)']

MJ_ini = []
CO2_ini = []
Count1 = 0
Count2 = 1

for i in range(Number_parameters):# -------------------------------- Parameter
    MJ_result = []
    CO2_result = [] 
    Scenario3 = Scenario2.copy()

    for j in range(len(Level)): # ------------------------------------------- level ('med','low','high') 
        if i == 12:
            Scenario3.loc[i,'Select_Level'] = Freight_Truck[Count1]
            Count1 = Count1+1
        elif i == 14:
            Scenario3.loc[i,'Select_Level'] = Last_Mile_Truck[Count2]
            Count2 = Count2+1  
        elif ((i == 13) | (i == 17) | (i == 18)):
            Scenario3.loc[i,'Select_Level'] = Level_inv[j]
        else:        
            Scenario3.loc[i,'Select_Level'] = Level[j]
            if i == 10:
                Scenario3.loc[12,'Select_Level'] = 'Rail'
        MJItem = []
        n=0
        for k in Years:#-------------------------------------------- Energy
            MJ_call = MJ(n,k,option,MSA_op,Scenario3,Energy_option)
            MJ_result.append(MJ_call[8])
            MJItem.append(MJRatil_Ecommerce)
            n=n+1
            del MJRatil_Ecommerce
            del MJ_call
        l = 0
        for m in Years:# ------------------------------------------- Emissions
            CO2_call = CO2(l,m,option,MSA_op,Scenario3,Energy_option)
            CO2_result.append(CO2_call[8])
            l=l+1
            del CO2_call
            
    # Saving data
    MJ_ini.append(MJ_result)
    CO2_ini.append(CO2_result)
    # Deleting all values
    del MJ_result
    del CO2_result
    del Scenario3
    del MJItem
    # Restoring values
    j=0
#print(pd.DataFrame(MJ_ini))

# Print matrix
Columns_year_level = ['2020med','2030med', '2040med', 
                     '2050med','2020low', '2030low',
                     '2040low','2050low', '2020high',
                     '2030high','2040high', '2050high',]

Type = ['Weight per item','Weight of shipping package','Return Rate Online Purchase',
                       'Return Rate In-store Purchase','Average Cost per item','Average item volume',
                       'Packaging material (Retail)','Packaging material (Etail)',
                       'Level of Rush Shipments','%Air','Rail','Air Carrier Shipment','Type of road vehicle for FTL',
                       'Shipping efficiency','Type of vehicle for last mile','Data Energy Consumption',
                       'Data usage per item','Basket size in-store','Basket size online',
                       'Retail energy use','(Omni) Warehouse energy use','Etail Warehouse energy use']

Groups_Retail = ['Warehousing', 'Retail Store', 'Customer Transport', 'Computer usage', 'Inbound', 
                 'Packaging','Product Info']
Groups_Etail = ['Warehousing', 'Customer Transport', 'Computer usage', 'Last Mile', 'Inbound', 
                'Packaging', 'Product Info']
Groups_Both = ['Warehousing', 'Retail Store', 'Customer Transport', 'Computer usage', 'Last Mile', 'Inbound', 
                'Packaging', 'Product Info']
# ------------------------------------------------ Saving data ----------------------------------------------- #
MJ_matrix1 = pd.DataFrame(MJ_base)
MJ_matrix2 = pd.DataFrame(MJ_ini)
CO2_matrix1 = pd.DataFrame(CO2_base)
CO2_matrix2 = pd.DataFrame(CO2_ini)
# Reset index 
MJ_matrix1 = MJ_matrix1.reset_index(drop=True)
MJ_matrix2 = MJ_matrix2.reset_index(drop=True)
CO2_matrix1 = CO2_matrix1.reset_index(drop=True)
CO2_matrix2 = CO2_matrix2.reset_index(drop=True)
# Join results
MJ_matrix=pd.concat([MJ_matrix1,MJ_matrix2],axis=1)
MJ_matrix.columns = Columns_year_level
MJ_matrix['Type'] = Type

CO2_matrix=pd.concat([CO2_matrix1,CO2_matrix2],axis=1)
CO2_matrix.columns = Columns_year_level
CO2_matrix['Type'] = Type


# ----------------------------------------------- Tornado Diagram --------------------------------------------- #
def Tornado2(matrix,base,best,worst,title_tornado,group,n):
    
    # Data to %
    matrix[best] = 100*(matrix[base]-matrix[best])/matrix[base]
    matrix[worst] = 100*(matrix[worst]-matrix[base])/matrix[base]
    matrix[base] = 1-matrix[base]/matrix[base]
 
    # Subplots
    fig, (ax_group, ax_left, ax_right) = plt.subplots(ncols=3, figsize=(11, 6))
    custom_xlim = (0, max(matrix[worst]))
    
        # Additional Text in the left side
    ax_group.axis([0, 14, 0, 14])
    ax_group.axis('off')
    ax_group.text(-6.5, 1, group[6], style='italic', fontsize = 10, fontweight='bold')#bbox={'facecolor': 'black', 'alpha': 0.3, 'pad': 10}
    ax_group.text(-6.5, 5-n, group[5], style='italic', fontsize = 10, fontweight='bold')
    ax_group.text(-6.5, 7, group[4], style='italic', fontsize = 10, fontweight='bold')
    ax_group.text(-6.5, 9, group[3], style='italic', fontsize = 10, fontweight='bold')
    ax_group.text(-6.5, 11, group[2], style='italic', fontsize = 10, fontweight='bold')
    ax_group.text(-6.5, 12, group[1], style='italic', fontsize = 10, fontweight='bold')
    ax_group.text(-6.5, 13, group[0], style='italic', fontsize = 10, fontweight='bold')
    # draw vertical line and small segments 
    ax_group.plot([2.4, 2.4], [0, 13.8], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [13.8, 13.8], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [12.5, 12.5], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [11.5, 11.5], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [10.5-n, 10.5-n], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [8.5+n, 8.5+n], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [5.5-n, 5.5-n], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [4.5-n, 4.5-n], 'k-', lw=1)
    ax_group.plot([2.4, 3.0], [0.01, 0.01], 'k-', lw=2)
    # Left side
    ax_left.barh(matrix['Type'], matrix[best], label="Best")
    ax_left.set_xlim(custom_xlim)
    ax_left.invert_xaxis()
    
    # Right side
    ax_right.barh(matrix['Type'], matrix[worst], facecolor='r', label='Worst')
    ax_right.set_yticks([])
    ax_right.set_xlim(custom_xlim)
    
    # Set legend
    fig.legend(bbox_to_anchor=(0.95, 0.5), loc="lower center")
    
    # Eliminate space between subplots
    plt.subplots_adjust(wspace=0, hspace=0)
        
    # Save plot 
    imgdata = io.BytesIO()
    fig.savefig(imgdata, format='png', bbox_inches='tight') # save in memory
    # Load existing workbook
    wb = openpyxl.load_workbook(Path+'/Results/Tornado_City_{}_{}_{}_Energy_{}_Year_{}.xlsx'.format(MSA_op, PG, Title, Energy_option, Year_Tornado))
    ws = wb[title_tornado]
    # Add image to worksheet
    img = openpyxl.drawing.image.Image(imgdata)
    ws.add_image(img, 'G0') # Save in excel
    # Save workbook
    wb.save(Path+'/Results/Tornado_City_{}_{}_{}_Energy_{}_Year_{}.xlsx'.format(MSA_op, PG, Title, Energy_option, Year_Tornado))

    return(plt.show())

# ---------------------------------------------- Print figures for energy------------------------------------ # 
# Year analyzed
Analysis = ['Type', str(Year_Tornado)+'med', str(Year_Tornado)+'low', str(Year_Tornado)+'high']

# Title
termwidth, fillchar = 78, '='
print(' Total Energy per item (MJ/item) for each component '.center(termwidth, fillchar))
Scenario_option = str(option)
Matrix = MJ_matrix[Analysis]
Matrix_E = CO2_matrix[Analysis]
print(Matrix)

with pd.ExcelWriter(Path+'/Results/Tornado_City_{}_{}_{}_Energy_{}_Year_{}.xlsx'.format(MSA_op, PG, Title, Energy_option, Year_Tornado)) as writer:
    Matrix.to_excel(writer, index=True, sheet_name='MJ')
    Matrix_E.to_excel(writer, index=True, sheet_name='CO2')
    
# Call the tornado function for JM
if Scenario_option == "1":
    Retail_Matrix = Matrix.drop(index=[2,4,7,8,9,10,14,18,21])
    Retail_Fig2 = Tornado2(Retail_Matrix,Analysis[1],Analysis[2],Analysis[3],'MJ',Groups_Retail,0)
elif Scenario_option == "2" or Scenario_option == "3":
    Etail_Matrix = Matrix.drop(index=[3,4,6,10,17,19,20])
    Etail_Fig = Tornado2(Etail_Matrix,Analysis[1],Analysis[2],Analysis[3],'MJ', Groups_Etail,0.5)
elif Scenario_option == "4":
    Both_Matrix = Matrix.drop(index=[4])
    Both_Fig = Tornado2(Both_Matrix,Analysis[1],Analysis[2],Analysis[3],'MJ', Groups_Both, 0)
    
# ---------------------------------------------- Print figures for emissions---------------------------------- # 
# Title
termwidth, fillchar = 78, '='
print(' Total Emissions per item (gr CO2/item) for each component '.center(termwidth, fillchar))


# Call the tornado function for CO2
if Scenario_option == "1":
    Retail_Matrix_E = Matrix_E.drop(index=[2,4,7,8,9,10,14,18,21])
    Retail_FigE2 = Tornado2(Retail_Matrix_E,Analysis[1],Analysis[2],Analysis[3],'CO2',Groups_Retail,0)
elif Scenario_option == "2" or Scenario_option == "3":
    Etail_Matrix_E = Matrix_E.drop(index=[3,4,6,10,17,19,20])
    Etail_FigE = Tornado2(Etail_Matrix_E,Analysis[1],Analysis[2],Analysis[3],'CO2', Groups_Etail,0.5)
elif Scenario_option == "4":
    Both_Matrix = Matrix.drop(index=[4])
    Both_FigE = Tornado2(Both_Matrix,Analysis[1],Analysis[2],Analysis[3],'CO2',Groups_Both, 0)
# ------------------------------------------------------------------------------------------------------------ # 


## 5.2. Energy scenarios - Plot

In [ ]:
# ---------------------------------------- Energy scenarios - Plot ---------------------------------------------#
Number_parameters = len(Scenario1.Select_Level)
for s in range(Number_parameters):
    if s == 12:
        Scenario2.loc[s,'Select_Level'] = 'Heavy Duty Truck (25 ton cargo)'
    elif s == 14:
        Scenario2.loc[s,'Select_Level'] = 'Medium Truck (20,000 lb)'
    else:
        Scenario2.loc[s,'Select_Level'] = 'med'
MJ_accumulate = []
CO2_accumulate = []
Energy_scenarios = np.array([1,2,3])
for j in Energy_scenarios: # ------------------------------------------- level ('med','low','high') 
    MJ_result = []
    CO2_result = []
    MJItem = []
    n = 0
    l = 0
    Percentage_option = j
    for k in Years:#-------------------------------------------- Energy
        MJ_call = MJ(n,k,option,MSA_op,Scenario1,j)
        MJ_result.append(MJ_call[8])
        MJItem.append(MJRatil_Ecommerce)
        n=n+1
        del MJRatil_Ecommerce
        del MJ_call
    MJ_accumulate.append(MJ_result)
    del MJ_result
    for m in Years:# ------------------------------------------- Emissions
        CO2_call = CO2(l,m,option,MSA_op,Scenario1,j)
        CO2_result.append(CO2_call[8])
        l=l+1
        del CO2_call
    CO2_accumulate.append(CO2_result)

# ----------------------------------------------- Saving data - Energy -----------------------------------------#
# Title
termwidth, fillchar = 78, '='
print(' Total Energy per item (MJ/item) for each scenario '.center(termwidth, fillchar))

MJ_transpose = np.transpose(MJ_accumulate)
MJ_table = pd.DataFrame(MJ_transpose)
Columns_energy = ['BAU-MOVES', 'BAU-EIA', 'NZE50']
MJ_table.columns=Columns_energy
MJ_table['Year'] = Years
MJ_table.set_index('Year', inplace=True)
display(MJ_table)  

#-------------------------------------------- Plot the Energy along the time -----------------------------------#
#plot each series with custom colors and labels
plt.plot(MJ_table, label=Columns_energy)
# Set legend
plt.legend(bbox_to_anchor=(1.2, 0.5), loc="lower center")
#add axes labels and a title
plt.ylabel('MJ/item', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.title('Energy', fontsize=16)
#display plot
plt.show()


# ---------------------------------------------- Saving data - Emissions ---------------------------------------# 
# Title
termwidth, fillchar = 78, '='
print(' Total Emissions per item (gr CO2/item) for each scenario '.center(termwidth, fillchar))

CO2_transpose = np.transpose(CO2_accumulate)
CO2_table = pd.DataFrame(CO2_transpose)
Columns_energy = ['BAU-MOVES', 'BAU-EIA', 'NZE50']
CO2_table.columns=Columns_energy
CO2_table['Year'] = Years
CO2_table.set_index('Year', inplace=True)
display(CO2_table)  

#-------------------------------------------- Plot the Emssions along the time ---------------------------------#
#plot each series with custom colors and labels
plt.plot(CO2_table, label=Columns_energy)
# Set legend
plt.legend(bbox_to_anchor=(1.2, 0.5), loc="lower center")
#add axes labels and a title
plt.ylabel('gr CO2/item', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.title('Emissions', fontsize=16)
#display plot
plt.show()
#---------------------------------------------------------------------------------------------------------------#


#### 5.3. Multiple growth poulation with energy scenario plots 

In [ ]:
Scenario4 = Scenario1.copy()
Number_parameters = len(Scenario4.Select_Level)
for s in range(Number_parameters):
    if s == 12:
        Scenario4.loc[s,'Select_Level'] = 'Heavy Duty Truck (25 ton cargo)'
    elif s == 14:
        Scenario4.loc[s,'Select_Level'] = 'Medium Truck (20,000 lb)'
    else:
        Scenario4.loc[s,'Select_Level'] = 'med'
MJ_accumulate = []
MJ_accumulate_scenario = []
Energy_scenarios = np.array([1,2,3])
Market_Population = ['MM', 'MH', 'HM', 'HH'] 
for s in Market_Population:
    PG = s
    MSA_Market_Population(str(MSA_op))
    VMT_3(MSA_op)
    for j in Energy_scenarios: # ------------------------------------------- level ('med','low','high') 
        MJ_result = []
        CO2_result = []
        MJItem = []
        n = 0
        l = 0
        Percentage_option = j
        for k in Years:#-------------------------------------------- Energy
            MJ_call = MJ(n,k,option,MSA_op,Scenario4,j)
            MJ_result.append(MJ_call[8])
            MJItem.append(MJRatil_Ecommerce)
            n = n+1
            del MJRatil_Ecommerce
            del MJ_call
        MJ_accumulate.append(MJ_result)
        del MJ_result
        for m in Years:# ------------------------------------------- Emissions
            CO2_call = CO2(l,m,option,MSA_op,Scenario4,j)
            CO2_result.append(CO2_call[8])
            l=l+1
            del CO2_call
    MJ_accumulate_scenario.append(MJ_accumulate)
    
# Saving data
MJ_transpose = np.transpose(MJ_accumulate)
MJ_table = pd.DataFrame(MJ_transpose)
Columns_energy = ['BAU-MOVES', 'BAU-EIA', 'NZE50']
#MJ_table.columns=Columns_energy
MJ_table['Year'] = Years
MJ_table.set_index('Year', inplace=True)
display(MJ_table)  

#-------------------------------------------- Plot the Energy along the time -----------------------------------#
# Subplots
fig, ax = plt.subplots(2, 2)
# plot
ax[0,0].plot(MJ_table.loc[:,0:2], label=Columns_energy)
ax[0,1].plot(MJ_table.loc[:,3:5])
ax[1,0].plot(MJ_table.loc[:,6:8])
ax[1,1].plot(MJ_table.loc[:,9:11])

#add legend
fig.legend(bbox_to_anchor=(1.1, 0.2), loc="lower center")
#add axes labels and a title
for ax in ax.flat:
    ax.set(xlabel='Year', ylabel='MJ/item')
fig.suptitle('Energy scenarios per future population growth', fontsize=16)
#display plot
fig.tight_layout()
plt.show()